## Preamble

In [ ]:
import numpy as np

import scipy as sp
import scipy.stats as stats
from scipy.stats import linregress,t
from scipy.optimize import curve_fit

import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from matplotlib import rc
plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=32) 
plt.rc('xtick', labelsize=32)
plt.rc('text.latex', preamble=r'\usepackage{color}')

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

import json

import os
import math
from tqdm import tqdm

import pandas as pd
import scikit_posthocs as skp
import statsmodels.stats.multitest as smm
from scipy.stats import norm

In [ ]:
# Plotting color
Sham = True
ColLS = True
C = np.array([[166,206,227],
[31,120,180],
[178,223,138],
[51,160,44],
[251,154,153],
[227,26,28],
[253,191,111],
[255,127,0],
[202,178,214],
[106,61,154],
[255,255,153],
            [169,169,169],
             [128,128,128]])
C = C/255.
ColorDict = {'mean':0,'median':1,'Control': 1,'CamKII':3,'Calcineurin':7,'CHX':5,'Anisomycin':9,'Sham':-1,'CamKII_alt':3,'CHX_alt':5,}

labelDict = {'CamKII':'AIP','Calcineurin':'FK506','Control':'Control','Sham':'Sham','CHX':'CHX','Anisomycin':'Anisomycin'}

spineplot = np.arange(1,16,0.5)
lw    = 4
fs    = 20

Fulltimes = [-15,-10,-5,2,10,20,30,40]

tmin    =  0
tmax    =  40
tpts    =  tmax*1000+1
dt     = (tmax-tmin)/tpts
tvec  = np.linspace(tmin,tmax,tpts)

DatDir = './DataProcessed/'

In [ ]:
def PreSortData(RealDat,Flag):
    
    Pot = []
    for i,d in enumerate(RealDat):
        if(abs((d[3]-d[:3].mean())/d[:3].std())>1.96 and d[3]-d[:3].mean()>0):
            Pot.append(Flag)
        else:
            Pot.append(not Flag)
    return np.delete(RealDat,Pot,axis=0)

def PreSortData2(RealDat,Flag):
    
    Pot = []
    for i,d in enumerate(RealDat):
        if(abs((d[3]-d[:3].mean())/d[:3].std())>1.96 and d[3]-d[:3].mean()>0):
            Pot.append(Flag)
        else:
            Pot.append(not Flag)
    return np.delete(RealDat,Pot,axis=0),Pot

def LoadOneSpine(Dir,Filter=True):

    Syn_a_arr = []
    for d in tqdm(os.listdir(Dir)):
        try:
            with open(Dir+d+'/Spine/Synapse_l.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
        except Exception as e:
            pass
            
    dists = []
    means = []
    lmeans = []
    for Syns in Syn_a_arr:
        d = []
        m = []
        lm = []
        for S in Syns:
            d.append(S["distance"])
            m.append(S["mean"])
            lm.append(S["local_bg"])
        dists.append(d)
        means.append(m)
        lmeans.append(lm)
    dists = np.array(dists)
    means = np.array(means)
    lmeans = np.array(lmeans)
    means = np.squeeze(means)
    lmeans = np.squeeze(lmeans)
    means = means-lmeans
    
    if(Filter):
        meansS = PreSortData(means,False)
    else:
        meansS = means
    meansS = (meansS.T/(meansS[:,:3].mean(axis=-1)))
    
    return meansS

def LoadSevenSpine(Dir,Flag=True):
    Syn_a_arr = []
    for d in tqdm(os.listdir(Dir)):
        try:
            with open(Dir+d+'/Spine/Synapse_l.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
        except Exception as e:
            print(e)
            
    dists = []
    means = []
    lmeans = []
    for Syns in Syn_a_arr:
        d = []
        m = []
        lm = []
        for S in Syns:
            d.append(S["distance"])
            m.append(S["mean"])
            lm.append(S["local_bg"])
        dists.append(d)
        means.append(np.array(m)-np.array(lm))

    middle = [np.array(m)[np.argsort(d)[3]] for m,d in zip(means,dists)]
    middleS = np.vstack(middle).squeeze()
    
    extreme = [np.array(m)[np.argsort(d)[[0,-1]]] for m,d in zip(means,dists)]
    extremeS = np.vstack(extreme).squeeze()
    if(Flag):
        middleS = PreSortData(middleS,False)
        extremeS = PreSortData(extremeS,False)
    middleS = (middleS.T/(middleS[:,:3].mean(axis=-1)))
    extremeS = (extremeS.T/(extremeS[:,:3].mean(axis=-1)))

    return [middleS,extremeS],[np.array(d)-min(d) for d in dists]

def LoadSevenSpine2(Dir,Flag=True):
    Syn_a_arr = []
    for d in tqdm(os.listdir(Dir)):
        try:
            with open(Dir+d+'/Spine/Synapse_l.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
        except Exception as e:
            print(e)
            pass
            
    dists = []
    means = []
    lmeans = []
    for Syns in Syn_a_arr:
        d = []
        m = []
        lm = []
        for S in Syns:
            d.append(S["distance"])
            m.append(S["mean"])
            lm.append(S["local_bg"])
        dists.append(d)
        means.append(np.array(m)-np.array(lm))

    middle = [np.array(m)[np.argsort(d)[[2,3,4]]] for m,d in zip(means,dists)]
    middleS = np.vstack(middle).squeeze()
    
    extreme = [np.array(m)[np.argsort(d)[[0,1,-2,-1]]] for m,d in zip(means,dists)]
    extremeS = np.vstack(extreme).squeeze()
    if(Flag):
        middleS = PreSortData(middleS,False)
        extremeS = PreSortData(extremeS,False)
    middleS = (middleS.T/(middleS[:,:3].mean(axis=-1)))
    extremeS = (extremeS.T/(extremeS[:,:3].mean(axis=-1)))

    return [middleS,extremeS],[np.array(d)-min(d) for d in dists]

def LoadSevenSpine3(Dir,Flag=True):
    Syn_a_arr = []
    for d in tqdm(os.listdir(Dir)):
        try:
            with open(Dir+d+'/Spine/Synapse_l.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
        except Exception as e:
            print(e)
            
    dists = []
    means = []
    lmeans = []
    for Syns in Syn_a_arr:
        d = []
        m = []
        lm = []
        for S in Syns:
            d.append(S["distance"])
            m.append(S["mean"])
            lm.append(S["local_bg"])
        dists.append(d)
        means.append(np.array(m)-np.array(lm))
    means = np.vstack(means).squeeze()
    meansS = (means.T/(means[:,:3].mean(axis=-1))).T
    p = None
    if(Flag):
        meansS,p = PreSortData2(meansS,False)
    
    return np.array(meansS),[np.array(d) for d in dists],p


def LoadThreeSpine(Dir,Flag):
    Syn_a_arr = []
    for d in tqdm(os.listdir(Dir)):
        try:
            with open(Dir+d+'/Spine/Synapse_l.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
            print(d)
        except Exception as e:
            print(e)
    
    dists = []
    means = []
    lmeans = []
    for Syns in Syn_a_arr:
        d = []
        m = []
        lm = []
        for S in Syns:
            d.append(S["distance"])
            m.append(S["mean"])
            lm.append(S["local_bg"])
        dists.append(d)
        means.append(m)
        lmeans.append(lm)
    dists = np.array(dists)
    means = np.array(means)
    lmeans = np.array(lmeans)

    means = np.squeeze(means)
    lmeans = np.squeeze(lmeans)
    means = means-lmeans
    
    middle = [means[i,np.argsort(dists,axis=1)[i,1]] for i in range(7)]
    lower = [means[i,np.argsort(dists,axis=1)[i,0]] for i in range(7)]
    upper = [means[i,np.argsort(dists,axis=1)[i,2]] for i in range(7)]
    middleS = np.array(middle)
    extremeS = np.vstack([lower,upper])
    if(Flag):
        middleS = PreSortData(middleS,False)
        extremeS = PreSortData(extremeS,False)
        
    middleS = (middleS.T/(middleS[:,:3].mean(axis=-1)))
    extremeS = (extremeS.T/(extremeS[:,:3].mean(axis=-1)))

    return [middleS,extremeS],(dists.T-dists.min(axis=-1)).T

def Load15Spine(Dir,Flag=True):
    Syn_a_arr = []
    for d in tqdm(os.listdir(Dir)):
        try:
            with open(Dir+d+'/Spine/Synapse_l.json', 'r') as fp: Syn_a_arr.append(json.load(fp))
        except Exception as e:
            print(e)
            
    dists = []
    means = []
    lmeans = []
    for Syns in Syn_a_arr:
        d = []
        m = []
        lm = []
        for S in Syns:
            d.append(S["distance"])
            m.append(S["mean"])
            lm.append(S["local_bg"])
        dists.append(d)
        means.append(np.array(m)-np.array(lm))
    middle = [np.array(m)[np.argsort(d)[[4,5,6,7,8,9,10]]] for m,d in zip(means,dists)]
    middleS = np.vstack(middle).squeeze()

    extreme = [np.array(m)[np.argsort(d)[[0,1,2,3,-4,-3,-2,-1]]] for m,d in zip(means,dists)]
    extremeS = np.vstack(extreme).squeeze()
    
    if(Flag):
        middleS = PreSortData(middleS,False)
        extremeS = PreSortData(extremeS,False)
    
    middleS = (middleS.T/(middleS[:,:3].mean(axis=-1)))
    extremeS = (extremeS.T/(extremeS[:,:3].mean(axis=-1)))

    return [middleS,extremeS],[np.array(d)-min(d) for d in dists]

def powerfunc(x, a, b,c):
    return a*x**b + c
def testfunc(x,a,b,c,d):
    return a + b*x**d/(c+x**d)
def logfunc(x,a,b,c):
    return a * np.log(b * x + c)
def func(x, a, c):
    return (a/(x))+c

def PlotSig(x1,x2,y,ydiff1=0.05,ydiff2=0.05,sig=True,stars=1,starpad=0.1,nsigVal = 'n.s.'):
    plt.plot([x1,x2],[y,y],c='k')
    if(sig):
        if(stars==1):
            plt.plot(x1/2+x2/2,y+ydiff1,marker=(6, 2, 0),c='k',ms=15)
        elif(stars==2):
            plt.plot(x1/2+x2/2-starpad,y+ydiff1,marker=(6, 2, 0),c='k',ms=15)
            plt.plot(x1/2+x2/2+starpad,y+ydiff1,marker=(6, 2, 0),c='k',ms=15)
    else:
        plt.text(x1/2+x2/2,y+ydiff1,nsigVal,fontsize=28,horizontalalignment='center')
    plt.plot([x1,x1],[y,y-ydiff2],c='k')
    plt.plot([x2,x2],[y,y-ydiff2],c='k')

def PlotSig2(x1,x2,y,ydiff1=0.05,ydiff2=0.05,):
    plt.plot([x1,x2],[y,y],c='k')
    plt.plot([x1,x1],[y,y-ydiff2],c='k')
    plt.plot([x2,x2],[y,y-ydiff2],c='k')

In [ ]:
class SpatialModel():
    def __init__(self,CParams,P0Params,gamma,zetaS,FParams,C0Params,tMax,xMax,stims):

        self.a1 = CParams[0]
        self.a2 = CParams[1]

        self.b1 = P0Params[0]
        self.b2 = P0Params[1]
        self.rho = P0Params[2]

        self.gamma = gamma

        self.zeta1 = zetaS[0]
        self.zeta2 = zetaS[1]
        self.nu   = FParams[0]
        self.phi  = FParams[1]

        self.lam = C0Params[0]
        self.mu  = C0Params[1]
        self.C_s = C0Params[2]
        self.C_d = C0Params[3]


        self.dx = 0.01
        self.x = np.arange(-xMax,xMax,self.dx)


        self.dt = 0.0025#min(0.001,self.dx*self.dx/(4*max(self.a1,self.b1)))
        self.tMax = tMax
        self.tvec = np.arange(0,tMax+self.dt,self.dt)

        Cd = self.initC(stims)
        Pd = 1-Cd
        self.C  = np.zeros_like(self.x) 
        PMod  = np.zeros_like(self.x) 
        
        self.P  = np.zeros_like(self.x)
        self.P0 = np.ones_like(self.x)
        
        self.S  = np.ones_like(self.x)

        for xs,c in zip(stims[0],Cd):
            self.C += (self.C_s+c)*np.exp(-SpineWidth*(self.x-xs)**2)
        for xs,c in zip(stims[0],Pd):
            PMod += (c)*np.exp(-SpineWidth*(self.x-xs)**2)
        
        self.P0 = self.rho*(self.P0-PMod)

        self.C_tot  = []
        self.P_tot  = []
        self.S_tot  = []
        self.P0_tot = []

    def CRHS(self):
    
        return self.a1*self.diff2(self.C)-self.a2*self.C

    def P0RHS(self):
        
        return self.b1*self.diff2(self.P0)-self.b2*self.C*self.P0

    def PRHS(self):
        
        return  self.b1*self.diff2(self.P) + self.b2*self.C*self.P0 - self.gamma*self.P

    def SRHS(self):
        return  self.zeta1*self.C + self.P*self.zeta2*self.F(self.S)

    def F(self,K):
        return -np.tanh(self.phi*(K-self.nu))
        
    def diff2(self,y):
        
        d2C = np.zeros_like(y)
        
        d2C[1:-1] = (np.roll(y,1)[1:-1]+np.roll(y,-1)[1:-1]-2*y[1:-1])/(self.dx*self.dx)
        
        return d2C

    def initC(self,stims,lam=1):
        x_stims = stims[0]
        t_stims = stims[1]
        C_shared     = np.ones_like(x_stims).astype(np.float64)
        n_stims = len(x_stims)
        if(n_stims>1):
            for i in range(n_stims):            
                mod = 0
                for j in range(n_stims):
                    if(i==j):
                        pass
                    else:
                        mod += (abs(x_stims[i]-x_stims[j])/(1+abs(x_stims[i]-x_stims[j])))**lam
                C_shared[i] = C_shared[i]*((mod+1)/(n_stims))
        return C_shared*self.C_d
    
    def Simulate(self):

        for _ in np.arange(0,self.tMax+self.dt,self.dt):
            RC   = self.CRHS()
            RP0  = self.P0RHS()
            RP   = self.PRHS()
            RS   = self.SRHS()

            self.C  = self.C  + self.dt*RC
            self.P0 = self.P0 + self.dt*RP0
            self.P  = self.P  + self.dt*RP
            self.S  = self.S  + self.dt*RS

            self.C_tot.append(self.C)
            self.P0_tot.append(self.P0)
            self.P_tot.append(self.P)
            self.S_tot.append(self.S)

        self.C_tot  = np.array(self.C_tot)
        self.P0_tot = np.array(self.P0_tot)
        self.P_tot  = np.array(self.P_tot)
        self.S_tot  = np.array(self.S_tot)


In [ ]:
def initC(stims,C_d,lam=1):
    x_stims = stims[0]
    t_stims = stims[1]
    C_shared     = np.ones_like(x_stims).astype(np.float64)
    n_stims = len(x_stims)
    if(n_stims>1):
        for i in range(n_stims):            
            mod = 0
            for j in range(n_stims):
                if(i==j):
                    pass
                else:
                    mod += (abs(x_stims[i]-x_stims[j])/(1+abs(x_stims[i]-x_stims[j])))
            C_shared[i] = C_shared[i]*((mod+1)/(n_stims))
    return C_shared*C_d

def RunSimul2(cPar,p0Par,pPar,zeta,phi,nu,stims):
    dx = 0.01
    x = np.arange(-2,2,dx)
    rho = 0.5

    C1  = np.zeros_like(x) 
    C  = np.zeros_like(x) 
    PMod  = np.zeros_like(x) 

    Cd = initC(stims,2)
    Pd = 1-Cd
    for xs,c in zip(stims[0],Cd):
        C1 += (Cs)*np.exp(-1000*(x-xs)**2)
        C += (Cs+c)*np.exp(-1000*(x-xs)**2)
        
            

    P0 = np.ones_like(x)

    for xs,c in zip(stims[0],Pd):
        PMod += (c)*np.exp(-1000*(x-xs)**2)

    P0 = rho*(P0-PMod)

    dt = dx*dx/(4*max(pPar[0],cPar[0]))
    Tmax = 40
    tvec = np.arange(0,Tmax+dt,dt)

    P = np.zeros_like(x)
    S = np.ones_like(x)

    C_tot  = []
    P0_tot = []
    P_tot  = []
    S_tot  = []
    for t in tqdm(np.arange(0,Tmax+dt,dt)):
        RC = CRHS(C,dx,cPar)
        RP0 = P0RHS(P0,C,dx,p0Par)
        RP   = PRHS(P,P0,C,dx,pPar)
        RS  = SRHS(C,direc(S,nu,phi)*P,zeta)
        C  = C  + dt*RC
        P0 = P0 + dt*RP0
        P  = P  + dt*RP
        S  = S  + dt*RS
        C_tot.append(C)
        P0_tot.append(P0)
        P_tot.append(P)
        S_tot.append(S)
    C_tot  = np.array(C_tot)
    P0_tot = np.array(P0_tot)
    P_tot  = np.array(P_tot)
    S_tot  = np.array(S_tot)

    return S_tot

def CRHS(C,dx,CParams):
    
    a1 = CParams[0]
    a2 = CParams[1]
    
    return a1*diff2(C,dx)-a2*C

def P0RHS(P0,C,dx,P0Params):
    b1 = P0Params[0]
    b2 = P0Params[1]
    
    return b1*diff2(P0,dx)-b2*C*P0

def PRHS(P,P0,C,dx,PParams):
    
    c1 = PParams[0]
    c2 = PParams[1]
    c3 = PParams[2]
    
    return  c1*diff2(P,dx) - c2*P + c3*C*P0

def SRHS(C,P,zeta):
    return 2*C + P*zeta

def direc(P,nu,phi):
    return -np.tanh(phi*(P-nu))
    
def diff2(y,dx):
    
    d2C = np.zeros_like(y)
    
    d2C[1:-1] = (np.roll(y,1)[1:-1]+np.roll(y,-1)[1:-1]-2*y[1:-1])/(dx*dx)
    
    return d2C

def wR2(dat,pred):
    y     = dat.mean(axis=1)
    yvar = dat.var(axis=1)
    ymean = dat.mean()
    sum1 = 0
    sum2 = 0
    for i,t in enumerate([2,10,20,30,40]):
        p = pred[t*400]
        sum1 +=  (p-y[i])**2/yvar[i]
        sum2 +=  (ymean-y[i])**2/yvar[i]
    return np.round(1-sum1/sum2,3)
    
def NMSE(dat,pred):
    NMSE = []
    for i,t in enumerate([2,10,20,30,40]):
        p = pred[t*400]
        y = dat[i]
        NMSE.append((sum((p-y)**2)/(sum((y-y.mean())**2))))
    return np.round(np.mean(NMSE),3)

def expfunc(x,a,b,c):
    return a*np.exp(-b*x)+c

def expfuncArr(x,params):
    a = params[0]
    b = params[1]
    c = params[2]
    return a*np.exp(-b*x)+c

## Fig 1

### f

In [ ]:
x = np.arange(0,2,0.01)
thr = 1.4
plt.plot(x,-np.tanh(2*(x-thr)),lw = 5,c='k')
plt.axvline(thr,lw=3,ls='--',c='r')
plt.axvline(1,lw=3,ls='--',c='k')
plt.text(thr,1.1,r'\bf{$\nu$}',ha='center',fontsize=32)
plt.axvspan(0, thr, alpha=0.5, color='red')
plt.axvspan(thr, 2, alpha=0.5, color='blue')
plt.text(0.02,-0.5,r'\bf Potentiating',fontsize=28)
plt.text(1.7,-0.2,r'\bf Depressing',fontsize=32,c='w',rotation=-90)
plt.text(1.05,0.35,r'$\mathbf{F(S)}$',fontsize=32,rotation=-45,c='k')
plt.xticks([])
plt.xlim([0,2])
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### g

In [ ]:
stims   = np.array([[-0.63,-0.42,-0.21,0,0.21,0.42,0.63],[0,0,0,0,0,0,0]])
lam = 1
Cs = 0.1
Cd = 2
xMax = 1

dx = 0.01
x = np.arange(-xMax,xMax,dx)
xplot = 6.66*x

C1  = np.zeros_like(x) 
C2  = np.zeros_like(x) 

Cd = initC(stims,Cd,lam)

for xs,c in zip(stims[0],Cd):
    C1 += (Cs)*np.exp(-1000*(x-xs)**2)
    C2 += (c)*np.exp(-1000*(x-xs)**2)
    
fig,ax = plt.subplots(figsize=(12,3))
plt.axvline(stims[0,0]*6.66,lw=4,c='k',ls='--')
plt.axvline(0,lw=4,c='k',ls='-')
plt.plot(xplot,C2,lw=4,c='b',label='Contribution $C_d$')
plt.plot(xplot,C1,lw=4,c='r',label='Contribution $C_s$')
#for s in stims[0]:
#    plt.axvline(s*6.66,c='k',lw=4,ls='--',alpha=0.25)
plt.axhline(C2.max(),c='k',lw=4,ls='--',alpha=0.25)
plt.axhline(C2[100],c='k',lw=4,ls='--',alpha=0.25)
leg =plt.legend(fontsize=30,loc=3,ncol=3,bbox_to_anchor=(0.,1))
leg.get_frame().set_linewidth(0.0)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.get_ylim()
plt.xticks([])

In [ ]:
stims   = np.array([[-0.63,-0.42,-0.21,0,0.21,0.42,0.63],[0,0,0,0,0,0,0]])
lam = 1
Cs = 0.1
Cd = 2
xMax = 1

dx = 0.01
x = np.arange(-xMax,xMax,dx)
xplot = 6.66*x

C1  = np.zeros_like(x) 
C2  = np.zeros_like(x) 

C3 = np.ones_like(x)
Cd = initC(stims,Cd,lam)
Cd = 1-Cd

for xs,c in zip(stims[0],Cd):
    C1 += (Cs)*np.exp(-1000*(x-xs)**2)
    C2 += (Cs+c)*np.exp(-1000*(x-xs)**2)
    
fig,ax = plt.subplots(figsize=(12,3))
plt.axvline(stims[0,0]*6.66,lw=4,c='k',ls='--')
plt.axvline(0,lw=4,c='k',ls='-')
plt.plot(xplot,0.5*(1-C2),lw=4,c='teal',label='Example init. $P_\mathrm{in}$')

#plt.plot(xplot,C1,lw=4,c='r',label='Contribution $C_s$')
plt.axhline(0.5*(1-C2).min(),c='k',lw=4,ls='--',alpha=0.25)
plt.axhline(0.5*(1-C2)[37],c='k',lw=4,ls='--',alpha=0.25)
leg =plt.legend(fontsize=30,loc=3,ncol=3,bbox_to_anchor=(0.,1))
leg.get_frame().set_linewidth(0.0)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.get_ylim()

### h

In [ ]:
cPar  = [0.001,1]
p0Par = [0.01,8]
pPar  = [0.01,0.01,8]
zeta  = 1
phi   = 1
nu    = 1.2
stims   = np.array([[-0.63,-0.42,-0.21,0,0.21,0.42,0.63],[0,0,0,0,0,0,0]])
Cs = 0.1
S = RunSimul2(cPar,p0Par,pPar,zeta,phi,nu,stims)

fig,ax = plt.subplots()
plt.plot(np.linspace(0,1,400),S[:400,200],lw=1,c='gray')
plt.plot(np.linspace(0,1,400),S[:400,137],lw=1,c='gray',ls='--')
plt.plot(np.linspace(1,40,15601),S[400:,200],lw=3,c='k',label='middle')
plt.plot(np.linspace(1,40,15601),S[400:,137],lw=3,c='k',ls='--',label='edge')
plt.legend(fontsize=32,frameon=False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.yticks(rotation=90,ha='right',va='center')

## Fig 2

In [ ]:
import json

try:
    with open('/Users/maximilianeggl/Dropbox/PostDoc/HeteroPlast/Code/SpatialPaper/MultiSpineModel/DataProcessed/3Spine/cell_1/Spine/Synapse_l.json', 'r') as file:
        data = json.load(file)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

In [ ]:
SpineDir = DatDir+'3Spine/'
m,d1 = LoadThreeSpine(SpineDir,False)
m[1] = PreSortData(m[1].T,False).T
mtot = np.hstack(m)
mtot3 = np.copy(mtot)
SpineDir = DatDir+'3SpineS/'
mS,d2 = LoadThreeSpine(SpineDir,False)

In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit3.npy')

a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_three = np.array([[-0.13,0,0.13],[0,0,0]])

SpineWidth = 1000
SM3 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM3.Simulate()
xvec = SM3.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]

Min3_m = SM3.S_tot[:,stimlocs_three[1]]
Min3_e = SM3.S_tot[:,stimlocs_three[0]]

stims_three = np.array([[-0.25,0,0.25],[0,0,0]])

SpineWidth = 1000
SM3 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM3.Simulate()
xvec = SM3.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]

Max3_m = SM3.S_tot[:,stimlocs_three[1]]
Max3_e = SM3.S_tot[:,stimlocs_three[0]]

stims_three = np.array([[-0.19,0,0.19],[0,0,0]])

SpineWidth = 1000
SM3 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM3.Simulate()
xvec = SM3.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]

Mid3_m = SM3.S_tot[:,stimlocs_three[1]]
Mid3_e = SM3.S_tot[:,stimlocs_three[0]]




### d

In [ ]:
plt.plot(SM3.tvec[:400],(SM3.S_tot[:400,stimlocs_three[0]]
                   +SM3.S_tot[:400,stimlocs_three[1]]+SM3.S_tot[:400,stimlocs_three[2]])/3,lw=2,ls='--')
plt.plot(SM3.tvec[400:],(SM3.S_tot[400:,stimlocs_three[0]]
                   +SM3.S_tot[400:,stimlocs_three[1]]+SM3.S_tot[400:,stimlocs_three[2]])/3,lw=5,
         label='Model fit',c=C[ColorDict['Control']])
ax = plt.gca()
_,caps,_ = ax.errorbar([2,10,20,30,40],mtot[3:].mean(axis=1),stats.sem(mtot[3:],axis=1),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Expt. spines',alpha=0.5
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
leg = ax.legend(framealpha=0.5,fontsize=32)
leg.get_frame().set_linewidth(0.0)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40],[2,10,20,30,40])
plt.text(18.2,1.9,'NMSE='+str(NMSE(mtot[3:],(SM3.S_tot[:,stimlocs_three[0]]+SM3.S_tot[:,stimlocs_three[1]]+SM3.S_tot[:,stimlocs_three[2]])/3)),fontsize=28)
plt.text(18.2,1.65,r'$R_w^2$='+str(wR2(mtot[3:],(SM3.S_tot[:,stimlocs_three[0]]+SM3.S_tot[:,stimlocs_three[1]]+SM3.S_tot[:,stimlocs_three[2]])/3)),fontsize=28)


### e

In [ ]:
plt.plot(SM3.tvec[:400],Mid3_m[:400],color=C[ColorDict['Control']],lw=2,ls='--')
plt.plot(SM3.tvec[:400],Mid3_e[:400],color=C[ColorDict['CamKII']],lw=2,ls='--')
plt.fill_between(SM3.tvec[400:],Max3_m[400:],Min3_m[400:],color=C[ColorDict['Control']],alpha=.25)
plt.plot(SM3.tvec[400:],Mid3_m[400:],color=C[ColorDict['Control']],lw=3)

plt.fill_between(SM3.tvec[400:],Max3_e[400:],Min3_e[400:],color=C[ColorDict['CamKII']],alpha=.25)
plt.plot(SM3.tvec[400:],Mid3_e[400:],color=C[ColorDict['CamKII']],lw=3)

ax = plt.gca()

x1, x2, y1, y2 = 1, 10, 1.2, 3.1  # subregion of the original image
axins = ax.inset_axes(
    [0.5, 0.5, 0.47, 0.47],
    xlim=(x1, x2), ylim=(y1, y2), xticklabels=[], yticklabels=[])
axins.fill_between(SM3.tvec[400:], Max3_m[400:], Min3_m[400:], color=C[ColorDict['Control']], alpha=.25)
axins.plot(SM3.tvec[400:], Mid3_m[400:], color=C[ColorDict['Control']],lw=3)
axins.xaxis.set_ticks_position('none') 
axins.yaxis.set_ticks_position('none') 
axins.fill_between(SM3.tvec[400:], Max3_e[400:], Min3_e[400:], color=C[ColorDict['CamKII']], alpha=.25)
axins.plot(SM3.tvec[400:], Mid3_e[400:], color=C[ColorDict['CamKII']],lw=3)
rect = (x1, y1, x2 - x1, y2 - y1)

box = ax.indicate_inset(rect, edgecolor="black", alpha=1,lw=2)

cp1 = ConnectionPatch(xyA=(x2, y1), xyB=(0, 0), axesA=ax, axesB=axins,
                      coordsA="data", coordsB="axes fraction", lw=1.5, ls=":")
cp2 = ConnectionPatch(xyA=(x2, y2), xyB=(0, 1), axesA=ax, axesB=axins,
                      coordsA="data", coordsB="axes fraction", lw=1.5, ls=":")

ax.add_patch(cp1)
ax.add_patch(cp2)
ax.set_ylim((0.5548009922104938, 3.2444933533858356))
ax.set_xticks([2,10,20,30,40],[2,10,20,30,40])
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
axins.spines["right"].set_visible(True)
axins.spines["top"].set_visible(True)


### f

In [ ]:
fig,ax = plt.subplots()
_,caps,_ = ax.errorbar([-15,-10,-5,2,10,20,30,40],m[0].mean(axis=1),stats.sem(m[0],axis=1),
             capsize=5,lw=3,marker='_',ecolor=C[ColorDict['Control']],
             c=C[ColorDict['Control']],elinewidth=2,ms=10,mew=3,capthick=1
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
    
_,caps,_ = ax.errorbar([-15,-10,-5,2,10,20,30,40],m[1].mean(axis=1),stats.sem(m[1],axis=1),
             capsize=5,lw=3,marker='_',ecolor=C[ColorDict['CamKII']],
             c=C[ColorDict['CamKII']],elinewidth=2,ms=10,mew=3,capthick=1
            )   
for cap in caps:
    cap.set_markeredgewidth(1)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([-15,-10,-5,2,10,20,30,40],[-15,-10,-5,2,10,20,30,40],rotation=45)


### g

In [ ]:
plt.bar([0],np.mean(m[0][3]),yerr=stats.sem(m[0][3]),width=0.25,color=C[ColorDict['Control']])
plt.scatter(0.*np.ones_like(m[0][3]),m[0][3],facecolors='none', edgecolors='k')
plt.bar([0.25],np.mean(m[1][3]),yerr=stats.sem(m[1][3]),width=0.25,color=C[ColorDict['CamKII']])
plt.scatter(0.25*np.ones_like(m[1][3]),m[1][3],facecolors='none', edgecolors='k')
print(stats.ttest_ind(m[0][3],mS[0][3]))
PlotSig(0,1,6.3,0.2,0.1,stars=1)

print(stats.ttest_ind(m[1][3],mS[1][3]))
PlotSig(0.25,1.25,5.8,0.2,0.1,)

comp_data = [m[0][3],mS[0][3],m[1][3],mS[1][3]]

ps = skp.posthoc_ttest(comp_data,p_adjust = 'fdr_bh')

print('='*20)
print('with correction')
print(np.array(ps)[0,1])
print(np.array(ps)[2,3])
print('='*20)
plt.bar([1],np.mean(mS[0][3]),yerr=stats.sem(mS[0][3]),width=0.25,color=C[ColorDict['Sham']])
plt.scatter(1*np.ones_like(mS[0][3]),mS[0][3],facecolors='none', edgecolors='k')
plt.bar([1.25],np.mean(mS[1][3]),yerr=stats.sem(mS[1][3]),width=0.25,color=C[ColorDict['Sham']-1])
plt.scatter(1.25*np.ones_like(mS[1][3]),mS[1][3],facecolors='none', edgecolors='k')
plt.xticks([0.125,1.125],['Stimulated','Sham'],fontsize=32)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)



### h

In [ ]:
plt.bar([0],np.mean(m[0][-1]),yerr=stats.sem(m[0][-1]),width=0.25,color=C[ColorDict['Control']])
plt.scatter(0*np.ones_like(m[0][-1]),m[0][-1],facecolors='none', edgecolors='k')
plt.bar([0.25],np.mean(m[1][-1]),yerr=stats.sem(m[1][-1]),width=0.25,color=C[ColorDict['CamKII']])
plt.scatter(0.25*np.ones_like(m[1][-1]),m[1][-1],facecolors='none', edgecolors='k')

PlotSig(0.25,1.25,2.2,0.15,0.1)

p1 = stats.ttest_ind(m[0][-1],mS[0][-1]).pvalue
p2 = stats.ttest_ind(m[1][-1],mS[1][-1]).pvalue
print(stats.ttest_ind(m[0][-1],mS[0][-1]))
print(stats.ttest_ind(m[1][-1],mS[1][-1]))

comp_data = [m[0][-1],mS[0][-1],m[1][-1],mS[1][-1]]


# Apply FDR correction (Benjamini-Hochberg)
rejected, p_values_corrected = smm.multipletests([p1,p2], alpha=0.05, method='fdr_bh')[:2]

print(p_values_corrected)
plt.bar([1],np.mean(mS[0][-1]),yerr=stats.sem(mS[0][-1]),width=0.25,color=C[ColorDict['Sham']])
plt.scatter(1*np.ones_like(mS[0][-1]),mS[0][-1],facecolors='none', edgecolors='k')
plt.bar([1.25],np.mean(mS[1][-1]),yerr=stats.sem(mS[1][-1]),width=0.25,color=C[ColorDict['Sham']-1])
plt.scatter(1.25*np.ones_like(mS[1][-1]),mS[1][-1],facecolors='none', edgecolors='k')
plt.xticks([0.125,1.125],['Stimulated','Sham'],fontsize=32)
plt.ylim((0.0, 3.9795))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


## Fig 3

In [ ]:
SpineDir = DatDir+'/1Spine/'
Control = LoadOneSpine(SpineDir,False)

SpineDir = DatDir+'/1SpineFK506/'
Calc = LoadOneSpine(SpineDir,False)

SpineDir = DatDir+'/1SpineAIP/'
Cam = LoadOneSpine(SpineDir,False)
SpineDir = DatDir+'/1SpineS/'
Sham = LoadOneSpine(SpineDir,False)
mt1 = np.copy(Control)



In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit3.npy')


a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]*3.5
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]/2

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])

SpineWidth = 1000
SM1 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_one)
SM1.Simulate()

In [ ]:
C1 = SM1.zeta1*np.sum(SM1.C_tot[:,200])*SM1.dt
Pact1  = SM1.P_tot[:,200]*SM1.F(SM1.S_tot[:,200])
Pp1 = SM1.zeta2*np.sum(Pact1[Pact1>0])*SM1.dt
Pm1 = SM1.zeta2*np.sum(Pact1[Pact1<0])*SM1.dt

### d

In [ ]:
_,caps,_ = plt.errorbar(Fulltimes,np.nanmean(Control,axis=1),stats.sem(Control,axis=1,nan_policy='omit'),
             capsize=5,marker='_',ecolor=C[ColorDict['Control']],label='Control',
             c=C[ColorDict['Control']],elinewidth=1,ms=10,mew=3,capthick=1,lw=lw
            )   
_,caps,_ = plt.errorbar(Fulltimes,np.mean(Calc,axis=1),stats.sem(Calc,axis=1),
             capsize=5,marker='_',ecolor=C[ColorDict['Calcineurin']],
             c=C[ColorDict['Calcineurin']],elinewidth=1,ms=10,mew=3,capthick=1,lw=lw,label='FK506'
            )   
_,caps,_ = plt.errorbar(Fulltimes,np.mean(Cam,axis=1),stats.sem(Cam,axis=1),
             capsize=5,marker='_',ecolor=C[ColorDict['CamKII']],
             c=C[ColorDict['CamKII']],elinewidth=1,ms=10,mew=3,capthick=1,lw=lw,label='AIP'
            )   
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

# get handles
handles, labels = ax.get_legend_handles_labels()
# remove the errorbars
handles = [h[0] for h in handles]
# use them in the legend
leg = ax.legend(handles, labels, framealpha=0.5,fontsize=28,loc=1,bbox_to_anchor=(1.11,1.12))
leg.get_frame().set_linewidth(0.0)
ax.set_xticks([-15,-10,-5,2,10,20,30,40],[-15,-10,-5,2,10,20,30,40],rotation=45)



### e

In [ ]:
plt.bar([0],np.mean(Control[3]),yerr=stats.sem(Control[3]))
plt.scatter(0*np.ones_like(Control[3]),Control[3],facecolors='none', edgecolors='k')
plt.bar([1],np.mean(Calc[3]),yerr=stats.sem(Calc[3]))
plt.scatter(1*np.ones_like(Calc[3]),Calc[3],facecolors='none', edgecolors='k')
plt.bar([2],np.mean(Cam[3]),yerr=stats.sem(Cam[3]))
plt.scatter(2*np.ones_like(Cam[3]),Cam[3],facecolors='none', edgecolors='k')
plt.bar([3],np.mean(Sham[3]),yerr=stats.sem(Sham[3]),color=C[ColorDict['Sham']])
plt.scatter(3*np.ones_like(Sham[3]),Sham[3],facecolors='none', edgecolors='k')
PlotSig(2,3,2.5,0.15,sig=False,nsigVal='$p=0.089$')
PlotSig(0,3,4.1,0.15,stars=1,starpad=0.1)
PlotSig(1,3,3.8,0.15,stars=1,starpad=0.1)

comp_data = [Control[3],Cam[3],Calc[3],Sham[3]]

ps = skp.posthoc_ttest(comp_data,p_adjust='fdr_bh')

print('='*20)
print(ps)
print('with correction')
print(np.array(ps)[0,-1])
print(np.array(ps)[2,-1])
print(np.array(ps)[0,2])
print(np.array(ps)[1,3])
print('='*20)

plt.xticks([0,1,2,3],['Control','FK506','AIP','Sham'],rotation=25,fontsize=36)


### f

In [ ]:
plt.bar([0],np.mean(Control[-1]),yerr=stats.sem(Control[-1]))
plt.scatter(0*np.ones_like(Control[-1]),Control[-1],facecolors='none', edgecolors='k')
plt.bar([1],np.mean(Calc[-1]),yerr=stats.sem(Calc[-1]))
plt.scatter(1*np.ones_like(Calc[-1]),Calc[-1],facecolors='none', edgecolors='k')
plt.bar([2],np.mean(Cam[-1]),yerr=stats.sem(Cam[-1]))
plt.scatter(2*np.ones_like(Cam[3]),Cam[-1],facecolors='none', edgecolors='k')
plt.bar([3],np.mean(Sham[-1]),yerr=stats.sem(Sham[-1]),color=C[ColorDict['Sham']])
plt.scatter(3*np.ones_like(Sham[-1]),Sham[-1],facecolors='none', edgecolors='k')

PlotSig(0,3,3.1,0.15)
PlotSig(1,3,2.7,0.15)

p1 = stats.ttest_ind(Control[-1],Sham[-1]).pvalue
p2 = stats.ttest_ind(Calc[-1],Sham[-1]).pvalue

rejected, p_values_corrected = smm.multipletests([p1,p2], alpha=0.05, method='fdr_bh')[:2]

print(p_values_corrected)

comp_data = [Control[-1],Cam[-1],Calc[-1],Sham[-1]]

print(stats.ttest_ind(Calc[-1],Sham[-1]).pvalue)

ps = skp.posthoc_ttest(comp_data)
print('='*20)
print('with correction')
print(np.array(ps)[0,-1])
print(np.array(ps)[2,-1])
print(np.array(ps)[0,2])
print(np.array(ps)[1,3])
print('='*20)

plt.xticks([0,1,2,3],['Control','FK506','AIP','Sham'],rotation=25,fontsize=36)

plt.ylim([0,4.4625])


### g

In [ ]:
plt.plot(SM1.tvec[:400],SM1.S_tot[:400,stimlocs_three[1]],c=C[ColorDict['Control']],lw=2,ls='--')
plt.plot(SM1.tvec[400:],SM1.S_tot[400:,stimlocs_three[1]],c=C[ColorDict['Control']],lw=5,label=r'Model pred.')
ax = plt.gca()
#_,caps,_ = ax.errorbar([2,10,20,30,40],np.nanmean(mt[3:],axis=1),stats.sem(mt[3:],axis=1,nan_policy='omit'),
#             capsize=5,ls='none',marker='_',ecolor='k',
#             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Avg. of expt.\n stim. spine',alpha=0.5
#            )   

_,caps,_ = ax.errorbar([2,10,20,30,40],np.nanmean(Control[3:],axis=1),stats.sem(Control[3:],axis=1,nan_policy='omit'),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Avg. of expt.\n stim. spine',alpha=0.5
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
leg = ax.legend(framealpha=0.5,fontsize=28,loc=1,bbox_to_anchor=(1,1.02),frameon=False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])
ax.set_ylim((0.8019776806368575, 2.8876038980241576))
plt.text(20.2,1.8,'NMSE='+str(NMSE(Control[3:], SM1.S_tot[:,stimlocs_three[1]])),fontsize=28)
plt.text(20.2,1.6,r'$R_w^2$='+str(wR2(Control[3:], SM1.S_tot[:,stimlocs_three[1]])),fontsize=28)


In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit1Calc.npy')


a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])

SpineWidth = 1000

SMCalc = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],
                   [lam,mu,Cs,Cd],tMax,xMax,stims_one)
SMCalc.Simulate()


### h

In [ ]:

p1 ,caps,_ = plt.errorbar([2,10,20,30,40],np.nanmean(Calc[3:],axis=1),stats.sem(Calc[3:],axis=1,nan_policy='omit'),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Avg. of expt.\n stim. spine \n FK506',alpha=0.5
            )   
p2 = plt.plot(SMCalc.tvec[:400],SMCalc.S_tot[:400,stimlocs_three[1]],c=C[ColorDict['Calcineurin']],lw=2,ls='--'
         )
p2 = plt.plot(SMCalc.tvec[400:],SMCalc.S_tot[400:,stimlocs_three[1]],c=C[ColorDict['Calcineurin']],lw=5,
         )
ax = plt.gca()
for cap in caps:
    cap.set_markeredgewidth(1)
leg = ax.legend(framealpha=0.0,fontsize=28,loc=1,bbox_to_anchor = (1.1,0.9))
#l1 = ax.legend([p1], ['Avg. of expt.\n stim. spine \n FK506'], loc=1,framealpha=0.5,fontsize=24)
l2 = ax.legend([p2[0]], [r'Model pred., higher $\frac{\zeta_1}{\zeta_2}$'], loc=1,framealpha=0.0,fontsize=28,
                bbox_to_anchor=(1.075,1.05))
leg.get_frame().set_linewidth(0.0)
l2.get_frame().set_linewidth(0.0)
ax.spines["right"].set_visible(False)
ax.add_artist(leg)
ax.add_artist(l2)
ax.spines["top"].set_visible(False)
ax.set_ylim((0.8019776806368575, 2.8876038980241576))
ax.set_xticks([2,10,20,30,40])
plt.text(2,1.1,'NMSE='+str(NMSE(Calc[3:], SMCalc.S_tot[:,stimlocs_three[1]])),fontsize=28)
plt.text(2,0.9,r'$R_w^2$='+str(wR2(Calc[3:], SMCalc.S_tot[:,stimlocs_three[1]])),fontsize=28)

In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit1Cam.npy')


a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]*1.05
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])

SpineWidth = 1000

SMCam = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],
                   [lam,mu,Cs,Cd],tMax,xMax,stims_one)
SMCam.Simulate()


### i

In [ ]:

p2 = plt.plot(SMCam.tvec[:400],SMCam.S_tot[:400,stimlocs_three[1]],c=C[ColorDict['CamKII']],lw=2,ls='--'
         )
plt.plot(SMCam.tvec[400:],SMCam.S_tot[400:,stimlocs_three[1]],c=C[ColorDict['CamKII']],lw=5,
         label=r'\hspace{-14pt}Model pred., lower $\frac{\zeta_1}{\zeta_2}$, $\nu$ \\ NSME='+str(NMSE(Cam[3:],SMCam.S_tot[:,stimlocs_three[1]]))+', $R_w^2$='+str(wR2(Cam[3:],SMCam.S_tot[:,stimlocs_three[1]])))
plt.axhline(1,ls='--',c=C[ColorDict['CamKII']-1],lw=5,label='Trivial solution, \n NSME='+str(NMSE(Cam[3:],np.ones(16001)))+', $R_w^2$='+str(wR2(Cam[3:],np.ones(16001))))
ax = plt.gca()
_,caps,_ = ax.errorbar([2,10,20,30,40],np.nanmean(Cam[3:],axis=1),stats.sem(Cam[3:],axis=1,nan_policy='omit'),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='expt. spine AIP',alpha=0.5
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
leg = ax.legend(framealpha=0.0,fontsize=24,loc=1,bbox_to_anchor=(1.1,1.05))
leg.get_frame().set_linewidth(0.0)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_ylim((0.8019776806368575, 2.8876038980241576))
ax.set_xticks([2,10,20,30,40])

## Fig 4 

In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit3.npy')

a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = 6*params[4]/7
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = 2.6*params[11]/7

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])
pix_to_mu = (0.63*2)/10.6
mu = 8/2
clust = pix_to_mu*mu
stims_seven = [np.round(clust/3,2),np.round(2*clust/3,2),np.round(clust,2)]
stims_seven   = np.array([[-s for s in stims_seven[::-1]]+[0]+stims_seven,[0,0,0,0,0,0,0]])


SpineWidth = 1000
stimlocs_seven = [np.argwhere(xvec == s)[0,0] for s in stims_seven[0]]


SM7 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_seven)
SM7.Simulate()

Min7_e = SM7.S_tot[:,stimlocs_seven[0]]
Min7_m = SM7.S_tot[:,stimlocs_seven[3]]

mu = 14.12/2
clust = pix_to_mu*mu
stims_seven = [np.round(clust/3,2),np.round(2*clust/3,2),np.round(clust,2)]
stims_seven   = np.array([[-s for s in stims_seven[::-1]]+[0]+stims_seven,[0,0,0,0,0,0,0]])

SpineWidth = 1000
stimlocs_seven = [np.argwhere(xvec == s)[0,0] for s in stims_seven[0]]


SM7 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_seven)
SM7.Simulate()

Max7_e = SM7.S_tot[:,stimlocs_seven[0]]
Max7_m = SM7.S_tot[:,stimlocs_seven[3]]

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])
stims_seven   = np.array([[-0.63,-0.42,-0.21,0,0.21,0.42,0.63],[0,0,0,0,0,0,0]])

SpineWidth = 1000
stimlocs_seven = [np.argwhere(xvec == s)[0,0] for s in stims_seven[0]]


SM7 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_seven)
SM7.Simulate()

Mid7_e = SM7.S_tot[:,stimlocs_seven[0]]
Mid7_m = SM7.S_tot[:,stimlocs_seven[3]]

In [ ]:
SpineDir = DatDir+'7Spine/'
mn,dn = LoadSevenSpine2(SpineDir,True)

m1 = mn[0].T
m2 = mn[1].T
mt7 = np.hstack(mn).T

mtot7 = np.copy(mt7)
SpineDirSham = DatDir+'7SpineS/'
mS,dS = LoadSevenSpine2(SpineDirSham,False)
mtotS = np.hstack(mS)

SpineDir = DatDir+'7SpineD/'
mD,dD = LoadSevenSpine2(SpineDir,True)
mtotD = np.hstack(mD)

### d

In [ ]:
plt.plot(SM7.tvec[:400],np.mean([SM7.S_tot[:400,s] for s in stimlocs_seven],axis=0),lw=2
        ,ls='--')
plt.plot(SM7.tvec[400:],np.mean([SM7.S_tot[400:,s] for s in stimlocs_seven],axis=0),lw=5
        ,label='Model pred.',color=C[ColorDict['Control']])
ax = plt.gca()
_,caps,_ = ax.errorbar([2,10,20,30,40],mt7[:,3:].mean(axis=0),stats.sem(mt7[:,3:],axis=0),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Expt. spines',alpha=0.5
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
    
    ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])
leg = ax.legend(framealpha=0.5,fontsize=32,loc=1,bbox_to_anchor=(1.1,1.05))
leg.get_frame().set_linewidth(0.0)
plt.text(6,1.2,'NMSE='+str(NMSE(mt7[:,3:].T,np.mean([SM7.S_tot[:,s] for s in stimlocs_seven],axis=0))),fontsize=28)
plt.text(6,1.05,r'$R^2_w$='+str(wR2(mt7[:,3:].T,np.mean([SM7.S_tot[:,s] for s in stimlocs_seven],axis=0))),fontsize=28)

### e

In [ ]:
plt.bar([0],np.mean(mt7[:,3]),yerr=stats.sem(mt7[:,3]),width=0.25,color=C[ColorDict['Control']])
plt.scatter(0.025*np.random.randn(*mt7[:,3].shape),mt7[:,3],facecolors='none', edgecolors='k',alpha=0.3)
plt.bar([0.3],np.mean(mt7[:,-1]),yerr=stats.sem(mt7[:,-1]),width=0.25,color=C[ColorDict['Control']])
plt.scatter(0.3+0.025*np.random.randn(*mt7[:,-1].shape),mt7[:,-1],facecolors='none', edgecolors='k',alpha=0.3)
plt.bar([1],np.mean(mtotS[3]),yerr=stats.sem(mtotS[3]),width=0.25,color=C[ColorDict['Sham']])
plt.bar([1.3],np.mean(mtotS[-1]),yerr=stats.sem(mtotS[-1]),width=0.25,color=C[ColorDict['Sham']])
plt.scatter(1+0.025*np.random.randn(*mtotS[3].shape),mtotS[3],facecolors='none', edgecolors='k',alpha=0.3)
plt.scatter(1.3+0.025*np.random.randn(*mtotS[-1].shape),mtotS[-1],facecolors='none', edgecolors='k',alpha=0.3)

PlotSig(0,1,4.2,0.2,stars=2,starpad=0.05)
print(stats.ttest_ind(mt7[:,3],mtotS[3]))

PlotSig(0.3,1.3,3.7,0.2)
print(stats.ttest_ind(mt7[:,-1],mtotS[-1]))

main_ticks = [0.15, 1.15]
main_labels = ['Stimulated', 'Sham']

# Sub ticks and labels
sub_ticks = [0., 0.3, 1., 1.3]
sub_labels = ['2 min', '40  min', '2  min', '40  min']
for i, sub_tick in enumerate(sub_ticks):
    plt.text(sub_tick, -0.1, sub_labels[i], ha='center', va='top', fontsize=21, rotation=0)

# Offset main ticks a bit lower
for tick, label in zip(main_ticks, main_labels):
    plt.text(tick, -0.5, label, ha='center', va='top', fontsize=32, weight='bold')

plt.xticks([])

ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

comp_data = [mt7[:,3],mtotS[3],mt7[:,-1],mtotS[-1]]

ps = skp.posthoc_ttest(comp_data,p_adjust='fdr_bh')
print('='*20)
print('with correction')
print(np.array(ps)[0,1])
print(np.array(ps)[1,2])
print(np.array(ps)[0,2])
print(np.array(ps)[2,3])
print('='*20)

### f

In [ ]:
plt.plot(SM7.tvec[:400],Mid7_m[:400],color=C[ColorDict['Control']],lw=1.5,ls='--')
plt.plot(SM7.tvec[:400],Mid7_e[:400],color=C[ColorDict['CamKII']],lw=1.5,ls='--')

plt.fill_between(SM7.tvec[400:],Min7_m[400:],Max7_m[400:],color=C[ColorDict['Control']],alpha=.25)
plt.plot(SM7.tvec[400:],Mid7_m[400:],color=C[ColorDict['Control']],lw=3)

plt.fill_between(SM7.tvec[400:],Min7_e[400:],Max7_e[400:],color=C[ColorDict['CamKII']],alpha=.25)
plt.plot(SM7.tvec[400:],Mid7_e[400:],color=C[ColorDict['CamKII']],lw=3)
ax = plt.gca()
_,caps,_ = ax.errorbar([2,10,20,30,40],m1[:,3:].mean(axis=0),stats.sem(m1[:,3:],axis=0),
             capsize=5,lw=3,ls='--',marker='_',ecolor=C[ColorDict['Sham']],
             c=C[ColorDict['Sham']],elinewidth=2,ms=10,mew=3,capthick=1
            )   
for cap in caps:
    cap.set_markeredgewidth(1)

_,caps,_ = ax.errorbar([2,10,20,30,40],m2[:,3:].mean(axis=0),stats.sem(m2[:,3:],axis=0),
             capsize=5,lw=3,ls='--',marker='_',ecolor=C[ColorDict['Sham']-1],
             c=C[ColorDict['Sham']-1],elinewidth=2,ms=10,mew=3,capthick=1
            )   
for cap in caps:
    cap.set_markeredgewidth(1)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])


PlotSig(20,20,1.6,sig=False,nsigVal = '$p=0.091$')
PlotSig(30,30,1.6,0.1)
PlotSig(40,40,1.6,0.1)

ax.set_ylim(0.9483729305222584,2.3)

plt.text(35,1.9,'NMSE='+str(NMSE(m2[:,3:].T,Mid7_e))+',\n $R_w^2$='+str(wR2(m2[:,3:].T,Mid7_e)),fontsize=28,c=C[ColorDict['CamKII']]
        ,horizontalalignment='center')
plt.text(0.4,1.15,'NMSE='+str(NMSE(m1[:,3:].T,Mid7_m)),fontsize=28,c=C[ColorDict['Control']])
plt.text(2,1,'$R_w^2$='+str(wR2(m1[:,3:].T,Mid7_m)),fontsize=28,c=C[ColorDict['Control']])


### g

In [ ]:
mind = []
for d in [dn]:
    for d2 in d:
        dt = np.sort(d2)
        for i in range(len(dt)):
            mind.append(min(abs(dt[i]-dt[(i-1)%len(dt)]),abs(dt[(i+1)%len(dt)]-dt[i])))
mindD = []
for d2 in dD:
    dt = np.sort(d2)
    for i in range(len(dt)):
        mindD.append(min(abs(dt[i]-dt[(i-1)%len(dt)]),abs(dt[(i+1)%len(dt)]-dt[i])))
        
plt.hist(mind,alpha=0.5,width=0.6,range=(0,max(mindD)),bins=20,color=C[ColorDict['Control']],label='Control, \n 7 spine')
plt.hist(mindD,alpha=0.5,width=0.6,range=(0,max(mindD)),bins=20,color=C[ColorDict['Calcineurin']],label='7 Distr.')

plt.axvline(np.mean(mind),color=C[ColorDict['Control']],lw=5)
plt.axvline(np.mean(mindD),color=C[ColorDict['Calcineurin']],lw=5)
ax = plt.gca()
leg = ax.legend(framealpha=0.5,fontsize=32,loc=1,bbox_to_anchor=[1.2,1.1],frameon=False)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit3.npy')
a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]*3.5
gamma = params[5]

nu = params[6]/1.1
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]/2

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])

SpineWidth = 1000
SM1d = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_one)
SM1d.Simulate()

params = np.load(DatDir+'ModelFit/ControlDistr.npy')

a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]/1.1
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_seven   = np.array([[-1.89,-1.26,-0.63,0,0.63,1.26,1.89],[0,0,0,0,0,0,0]])

SpineWidth = 1000
SM2 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_one)
SM2.Simulate()



### h

In [ ]:
fig,ax = plt.subplots()

_,caps,_ = ax.errorbar([-15,-10,-5,2,10,20,30,40],mt7.mean(axis=0),stats.sem(mt7,axis=0),
         capsize=5,lw=3,marker='_',ecolor=C[ColorDict['Control']],
         c=C[ColorDict['Control']],elinewidth=2,ms=10,mew=3,capthick=1,label='7 spine'
        )   

_,caps,_ = ax.errorbar([-15,-10,-5,2,10,20,30,40],mtotD.mean(axis=1),stats.sem(mtotD,axis=1),
         capsize=5,lw=3,marker='_',ecolor=C[ColorDict['Calcineurin']],
         c=C[ColorDict['Calcineurin']],elinewidth=2,ms=10,mew=3,capthick=1,
                       label='7 Distr. spine'
        )   



_,caps,_ = ax.errorbar([-15,-10,-5,2,10,20,30,40],Control.mean(axis=1),stats.sem(Control,axis=1),
         capsize=5,lw=3,marker='_',ecolor=C[ColorDict['Control']-1],
         c=C[ColorDict['Control']-1],elinewidth=2,ms=10,mew=3,capthick=1,
                       label='1 spine',ls='--'
        )   


handles, labels = ax.get_legend_handles_labels()
# remove the errorbars
handles = [h[0] for h in handles]
# use them in the legend
leg = ax.legend(handles, 
                labels,ncols=3, framealpha=0.5,fontsize=24,loc=1,
                bbox_to_anchor=(1.1,1.2),frameon=False,columnspacing=0.3,handlelength=0.6,handletextpad=0.1)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([-15,-10,-5,2,10,20,30,40],[-15,-10,-5,2,10,20,30,40],rotation=45)

plt.plot([2,2],[2.1,2.2],c='k')
plt.plot(2,2.4,marker=(6, 2, 0),c='k',ms=10)
plt.text(2,2.25,'p=0.02',ha='center',fontsize=18)

plt.plot([10,10],[2,2.1],c='k')
plt.text(10,2.15,'0.072',ha='center',fontsize=18)

plt.plot([20,20],[1.8,1.9],c='k')
plt.plot(20,2.1,marker=(6, 2, 0),c='k',ms=10)
plt.text(20,1.95,'p=0.049',ha='center',fontsize=18)

plt.plot([30,30],[1.8,1.9],c='k')
plt.text(30,1.95,'p=0.1',ha='center',fontsize=18)


plt.plot([40,40],[1.5+0.2,1.6+0.2],c='k')
plt.text(40,1.85,'p=0.02',ha='center',fontsize=18)
plt.plot(40,2.,marker=(6, 2, 0),c='k',ms=10)

plt.savefig('/Users/maximilianeggl/Dropbox/PostDoc/SpatialPaper/Rebuttal2Figs/Review2/Figures/7DistComp.pdf',format='pdf',bbox_inches='tight')

In [ ]:
plt.plot(SM2.tvec[:400],SM2.S_tot[:400,stimlocs_three[1]],c=C[ColorDict['Calcineurin']],lw=2,ls='--')
plt.plot(SM2.tvec[400:],SM2.S_tot[400:,stimlocs_three[1]],c=C[ColorDict['Calcineurin']],lw=5,label='Distr. pred. \n NMSE='+
         str(NMSE(mtotD[3:],SM2.S_tot[:,stimlocs_three[1]]))+', $R_w^2$='+str(wR2(mtotD[3:],SM2.S_tot[:,stimlocs_three[1]])))
plt.plot(SM2.tvec[:400],SM1.S_tot[:400,stimlocs_three[1]],c=C[ColorDict['Control']-1],lw=2,ls='--')
plt.plot(SM2.tvec[400:],SM1.S_tot[400:,stimlocs_three[1]],c=C[ColorDict['Control']-1],lw=5,label='1 spine pred. \n NMSE='+
         str(NMSE(mtotD[3:],SM1.S_tot[:,stimlocs_three[1]]))+', $R_w^2$='+str(wR2(mtotD[3:],SM1.S_tot[:,stimlocs_three[1]])),ls='--')

ax = plt.gca()
_,caps,_ = ax.errorbar([2,10,20,30,40],mtotD[3:].mean(axis=1),stats.sem(mtotD[3:],axis=1),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Avg. of expt. stim.\n spine, Distr.'
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
leg = ax.legend(framealpha=0.5,fontsize=24,loc=1,bbox_to_anchor=(1.,1.1),frameon=False,handlelength=0.8)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])


## Fig 5

In [ ]:
SpineDir = DatDir+'15Spine/'
m,d = Load15Spine(SpineDir,True)
mtot = np.hstack(m)
mtot15 = np.copy(mtot)
SpineDir = DatDir+'15SpineS/'
ms,ds = Load15Spine(SpineDir,False)
mtots = np.hstack(m)

In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit3.npy')
a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]/6
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]/15

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])
stims_seven   = np.array([[-0.63,-0.42,-0.21,0,0.21,0.42,0.63],[0,0,0,0,0,0,0]])
stims_fifteen = np.array([[-1.68,-1.44,-1.2,-0.96,-0.72,-0.48,-0.24,0,0.24,0.48,0.72,0.96,1.2,1.44,1.68],
                        [0]*15])
    
SpineWidth = 1000
stimlocs_seven = [np.argwhere(xvec == s)[0,0] for s in stims_seven[0]]
stimlocs_15 = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]


SM15 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_fifteen)
SM15.Simulate()

In [ ]:
SpineDir = DatDir+'15Spine/'
m,d,p = LoadSevenSpine3(SpineDir)
SType = []
for testd in d:
    stype = []
    for td in testd:
        if((testd-td <= 0).all() or (testd-td>=0).all()):
           stype.append('o')
        else:
            diff = testd-td
            p1 = diff[diff>0].min()
            p2 = diff[diff<0].max()
            if(p1>3 or p2 < -3):
                stype.append('e')
            else:
                stype.append('m')
    SType.append(stype)
SType = np.hstack(SType)

S = SType[[not p1 for p1 in p]]

mt15 = np.copy(m)

params = np.load(DatDir+'ModelFit/ControlFit3.npy')
a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]/4
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]/10

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_one   = np.array([[0],[0]])
dist = 0.17
side = [np.round(dist*i,2) for i in range(1,8)]
stims = np.round(-1*np.array(side[::-1]),2).tolist()+[0]+side
stims_fifteen = np.array([stims,
                        [0]*15])
    
SpineWidth = 1000
stimlocs_fifteen = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]
stimlocs_15 = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]


SM15 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_fifteen)
SM15.Simulate()

Min15_e = SM15.S_tot[:,stimlocs_fifteen[0]]
Min15_m = SM15.S_tot[:,stimlocs_fifteen[7]]

dist = 0.26
side = [np.round(dist*i,2) for i in range(1,8)]
stims = np.round(-1*np.array(side[::-1]),2).tolist()+[0]+side
stims_fifteen = np.array([stims,
                        [0]*15])
    
SpineWidth = 1000
stimlocs_fifteen = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]
stimlocs_15 = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]


SM15 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_fifteen)
SM15.Simulate()

Max15_e = SM15.S_tot[:,stimlocs_fifteen[0]]
Max15_m = SM15.S_tot[:,stimlocs_fifteen[7]]

stims_one   = np.array([[0],[0]])
stims_fifteen = np.array([[-1.68,-1.44,-1.2,-0.96,-0.72,-0.48,-0.24,0,0.24,0.48,0.72,0.96,1.2,1.44,1.68],
                        [0]*15])
    
SpineWidth = 1000
stimlocs_fifteen = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]
stimlocs_15 = [np.argwhere(xvec == s)[0,0] for s in stims_fifteen[0]]


SM15 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_fifteen)
SM15.Simulate()

Mid15_e = SM15.S_tot[:,stimlocs_fifteen[0]]
Mid15_m = SM15.S_tot[:,stimlocs_fifteen[7]]

### d

In [ ]:
plt.plot(SM15.tvec[:400],np.mean([SM15.S_tot[:400,s] for s in stimlocs_15],axis=0),lw=2,ls='--',color=C[ColorDict['Control']])
plt.plot(SM15.tvec[400:],np.mean([SM15.S_tot[400:,s] for s in stimlocs_15],axis=0),lw=5
        ,label='Model pred.')
ax = plt.gca()
_,caps,_ = ax.errorbar([2,10,20,30,40],mtot[3:].mean(axis=1),stats.sem(mtot[3:],axis=1),
             capsize=5,ls='none',marker='_',ecolor='k',
             c='k',elinewidth=2,ms=10,mew=3,capthick=1,label='Expt. spines',alpha=0.5
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
    
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])
leg = ax.legend(framealpha=0.5,fontsize=32,loc=4,bbox_to_anchor=(1,0.15),frameon=False)
plt.text(18.6,1.08,'NMSE='+str(NMSE(mtot[3:], np.mean([SM15.S_tot[:,s] for s in stimlocs_15],axis=0))),fontsize=28)
plt.text(18.5,1,'$R_w^2$='+str(wR2(mtot[3:], np.mean([SM15.S_tot[:,s] for s in stimlocs_15],axis=0))),fontsize=28)


### e

In [ ]:
plt.bar([0],np.mean(m[:,3]),yerr=stats.sem(m[:,3]),width=0.25,color=C[ColorDict['Control']])
plt.scatter(0.025*np.random.randn(*m[:,3].shape),m[:,3],facecolors='none', edgecolors='k',alpha=0.2)
plt.bar([0.3],np.mean(m[:,-1]),yerr=stats.sem(m[:,-1]),width=0.25,color=C[ColorDict['Control']])
plt.scatter(0.3+0.025*np.random.randn(*m[:,-1].shape),m[:,-1],facecolors='none', edgecolors='k',alpha=0.2)

plt.bar([1],np.mean(mtotS[3]),yerr=stats.sem(mtotS[3]),width=0.25,color=C[ColorDict['Sham']])
plt.scatter(1+0.025*np.random.randn(*mtotS[3].shape),mtotS[3],facecolors='none', edgecolors='k',alpha=0.2)
plt.bar([1.3],np.mean(mtotS[-1]),yerr=stats.sem(mtotS[-1]),width=0.25,color=C[ColorDict['Sham']])
plt.scatter(1.3+0.025*np.random.randn(*mtotS[-1].shape),mtotS[-1],facecolors='none', edgecolors='k',alpha=0.2)

PlotSig(0,1,4,0.2,stars=2,starpad=0.04)
print(stats.ttest_ind(m[:,3],mtotS[3]))

PlotSig(0.25,1.25,3.7,0.2)
print(stats.ttest_ind(m[:,-1],mtotS[-1]))

plt.text(-0.06,0.1,'2 min',rotation=90,fontsize=24,color='w')
plt.text(0.3-0.06,0.1,'40 min',rotation=90,fontsize=24,color='w')

plt.text(1-0.06,0.1,'2 min',rotation=90,fontsize=24,color='w')
plt.text(1.3-0.06,0.1,'40 min',rotation=90,fontsize=24,color='w')
plt.xticks([0.15,1.15],['Stimulated','Sham'],fontsize=38)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

comp_data = [m[:,3],mtotS[3],m[:,-1],mtotS[-1]]

ps = skp.posthoc_ttest(comp_data,p_adjust='fdr_bh')
print('='*20)
print('with correction')
print(np.array(ps)[0,1])
print(np.array(ps)[1,2])
print(np.array(ps)[0,2])
print(np.array(ps)[2,3])
print('='*20)


### f

In [ ]:
fig,ax = plt.subplots(figsize=(6.4,9.6))
plt.fill_between(SM15.tvec[400:],Min15_m[400:],Max15_m[400:],color=C[ColorDict['Control']],alpha=.25)
plt.plot(SM15.tvec[400:],Mid15_m[400:],color=C[ColorDict['Control']],label='Middle',lw=3)
plt.plot(SM15.tvec[:400],Mid15_m[:400],color=C[ColorDict['Control']],lw=1.5,ls='--')

plt.fill_between(SM15.tvec[400:],Min15_e[400:],Max15_e[400:],color=C[ColorDict['CamKII']],alpha=.25)
plt.plot(SM15.tvec[400:],Mid15_e[400:],color=C[ColorDict['CamKII']],label='Edge',lw=3)
plt.plot(SM15.tvec[:400],Mid15_e[:400],color=C[ColorDict['CamKII']],lw=1.5,ls='--')
ax = plt.gca()
m1 = m[S=='m']
_,caps,_ = ax.errorbar([2,10,20,30,40],m1[:,3:].mean(axis=0),stats.sem(m1[:,3:],axis=0),
             capsize=5,lw=3,ls='--',marker='_',ecolor=C[ColorDict['Sham']],
             c=C[ColorDict['Sham']],elinewidth=2,ms=10,mew=3,capthick=1,label='Expt.'
            )   
for cap in caps:
    cap.set_markeredgewidth(1)

m2 = m[np.array(S=='e')+np.array(S=='o')]
_,caps,_ = ax.errorbar([2,10,20,30,40],m2[:,3:].mean(axis=0),stats.sem(m2[:,3:],axis=0),
             capsize=5,lw=3,ls='--',marker='_',ecolor=C[ColorDict['Sham']-1],
             c=C[ColorDict['Sham']-1],elinewidth=2,ms=10,mew=3,capthick=1
            )   
for cap in caps:
    cap.set_markeredgewidth(1)
plt.text(2,1.12,'NMSE='+str(NMSE(m2[:,3:].T,Mid15_e))+', $R_w^2$='+str(wR2(m2[:,3:].T,Mid15_e)),fontsize=30,color=C[ColorDict['CamKII']])
plt.text(2,1.06,'NMSE='+str(NMSE(m1[:,3:].T,Mid15_m))+', $R_w^2$='+str(wR2(m1[:,3:].T,Mid15_m)),fontsize=30,color=C[ColorDict['Control']])

leg = ax.legend(framealpha=0.5,fontsize=32,frameon=False,ncol=3
                ,columnspacing=0.3,handlelength=0.8,handletextpad=0.1,loc=1,
                bbox_to_anchor=(1.1,0.4))
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])

plt.plot([30,30],[1.7,1.75],c='k')
plt.plot(30,1.78,marker=(6, 2, 0),c='k',ms=10)

plt.plot([40,40],[1.7,1.75],c='k')
plt.plot(39.3,1.78,marker=(6, 2, 0),c='k',ms=10)
plt.plot(40.7,1.78,marker=(6, 2, 0),c='k',ms=10)

print(stats.ttest_ind(m1[-2],m2[-2]).pvalue)
print(stats.ttest_ind(m1[-1],m2[-1]).pvalue)

p1 = stats.ttest_ind(m1[-2],m2[-2]).pvalue
p2 = stats.ttest_ind(m1[-1],m2[-1]).pvalue

rejected, p_values_corrected = smm.multipletests([p1,p2], alpha=0.05, method='fdr_bh')[:2]
print(p_values_corrected)


### g

In [ ]:
plt.plot(SM15.tvec[:400],SM15.C_tot[:400,stimlocs_fifteen[0]]-SM15.C_tot[:400,stimlocs_fifteen[7]],
         c=C[ColorDict['Control']],lw=2,ls='--')
plt.plot(SM15.tvec[400:],SM15.C_tot[400:,stimlocs_fifteen[0]]-SM15.C_tot[400:,stimlocs_fifteen[7]],
         c=C[ColorDict['Control']],lw=5,label='Difference between \n C at edge and middle')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.legend(ncols=2,fontsize=28,loc=5,frameon=False,handlelength=0.5,borderpad=0.0,
             handletextpad=0.5,columnspacing=0.5)


### h

In [ ]:

plt.plot(SM15.tvec[:400],zeta1*SM15.P_tot[:400,stimlocs_fifteen[0]],
         c= [1,0,0],lw=2,ls='--')
plt.plot(SM15.tvec[400:],zeta1*SM15.P_tot[400:,stimlocs_fifteen[0]],
         c= [1,0,0],lw=5,label='P - edge')
plt.plot(SM3.tvec[400:],zeta1*SM15.P_tot[400:,stimlocs_fifteen[7]],
         c= [1,0.5,0.5],lw=5,label='P - mid.',ls='--')
plt.plot(SM3.tvec[:400],zeta1*SM15.P_tot[:400,stimlocs_fifteen[7]],
         c= [1,0.5,0.5],lw=2,ls='--')
plt.legend(ncols=2,fontsize=28,loc=1,frameon=False,handlelength=0.5,borderpad=0.0,
             handletextpad=0.5,columnspacing=0.5)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


## Fig 6

### a

In [ ]:
_,caps,_ = plt.errorbar([-15,-10,-5,2,10,20,30,40],np.nanmean(Control,axis=1),stats.sem(Control,axis=1,nan_policy='omit'),
             capsize=5,marker='_',ecolor=C[ColorDict['Control']],
             c=C[ColorDict['Control']],lw=3,elinewidth=2,ms=10,mew=3,capthick=1,label='1 Spine'
            )   

_,caps,_ = plt.errorbar([-15,-10,-5,2,10,20,30,40],np.nanmean(mtot3,axis=1),stats.sem(mtot3,axis=1,nan_policy='omit'),
             capsize=5,marker='_',ecolor=C[ColorDict['Control']]+0.15,
            c=C[ColorDict['Control']]+0.15,lw=3,elinewidth=2,ms=10,mew=3,capthick=1,label='3 Spine',ls='--'
            )   

_,caps,_ = plt.errorbar([-15,-10,-5,2,10,20,30,40],np.nanmean(mt7,axis=0),stats.sem(mt7,axis=0,nan_policy='omit'),
             capsize=5,marker='_',ecolor=np.clip(C[ColorDict['Control']]+0.3,0,1),
            c=np.clip(C[ColorDict['Control']]+0.3,0,1),lw=3,elinewidth=2,ms=10,mew=3,capthick=1,label='7 Spine',ls='-.'
            )   

_,caps,_ = plt.errorbar([-15,-10,-5,2,10,20,30,40],np.nanmean(mtot15,axis=1),stats.sem(mtot15,axis=1,nan_policy='omit'),
             capsize=5,marker='_',ecolor=np.clip(C[ColorDict['Control']]+0.45,0,1),
            c=np.clip(C[ColorDict['Control']]+0.45,0,1),lw=3,elinewidth=2,ms=10,mew=3,capthick=1,label='15 Spine'
            )   
plt.legend(fontsize=28,frameon=False,loc = 1,bbox_to_anchor=(1.1,1.1))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])
plt.ylim((0.9076591483301499, 3.006948551593322))



### b

In [ ]:
plt.plot(SM1.tvec[:400],SM1.S_tot[:400,stimlocs_three[1]],c=C[ColorDict['Control']],lw=2,ls='--')
plt.plot(SM1.tvec[400:],SM1.S_tot[400:,stimlocs_three[1]],c=C[ColorDict['Control']],lw=5,label=r'1 Spine')
plt.plot(SM3.tvec[:400],(SM3.S_tot[:400,stimlocs_three[0]]
                   +SM3.S_tot[:400,stimlocs_three[1]]+SM3.S_tot[:400,stimlocs_three[2]])/3,lw=2,c=C[ColorDict['Control']]+0.15,
         ls='--')
plt.plot(SM3.tvec[400:],(SM3.S_tot[400:,stimlocs_three[0]]
                   +SM3.S_tot[400:,stimlocs_three[1]]+SM3.S_tot[400:,stimlocs_three[2]])/3,lw=5,c=C[ColorDict['Control']]+0.15,
         label='3 Spine',ls='--')

plt.plot(SM7.tvec[:400],np.mean([SM7.S_tot[:400,s] for s in stimlocs_seven],axis=0),lw=2,c=np.clip(C[ColorDict['Control']]+0.3,0,1),
        ls='--')
plt.plot(SM7.tvec[400:],np.mean([SM7.S_tot[400:,s] for s in stimlocs_seven],axis=0),lw=5,c=np.clip(C[ColorDict['Control']]+0.3,0,1),
        label='7 Spine',ls='-.')
plt.plot(SM15.tvec[:400],np.mean([SM15.S_tot[:400,s] for s in stimlocs_15],axis=0),lw=2,c=np.clip(C[ColorDict['Control']]+0.45,0,1),
          ls='--')
plt.plot(SM15.tvec[400:],np.mean([SM15.S_tot[400:,s] for s in stimlocs_15],axis=0),lw=5,c=np.clip(C[ColorDict['Control']]+0.45,0,1),
         label='15 Spine')
plt.legend(fontsize=28,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])
plt.gca().get_ylim()
plt.gca().get_xlim()

### c

In [ ]:
C1 = SM1.zeta1*np.sum(SM1.C_tot[:,200])*SM1.dt
Pact1  = SM1.P_tot[:,200]*SM1.F(SM1.S_tot[:,200])
Pp1 = SM1.zeta2*np.sum(Pact1[Pact1>0])*SM1.dt
Pm1 = SM1.zeta2*np.sum(Pact1[Pact1<0])*SM1.dt

C3 = SM3.zeta1*np.sum([SM3.C_tot[:,s] for s in stimlocs_three],axis=1).mean()*SM1.dt
Pact3  = np.mean([SM3.P_tot[:,s]*SM3.F(SM3.S_tot[:,s]) for s in stimlocs_three],axis=0)
Pp3 = SM3.zeta2*np.sum(Pact3[Pact3>0])*SM1.dt
Pm3 = SM3.zeta2*np.sum(Pact3[Pact3<0])*SM1.dt

C7 = SM7.zeta1*np.sum([SM7.C_tot[:,s] for s in stimlocs_seven],axis=1).mean()*SM1.dt
Pact7  = np.mean([SM7.P_tot[:,s]*SM7.F(SM7.S_tot[:,s]) for s in stimlocs_seven],axis=0)
Pp7 = SM7.zeta2*np.sum(Pact7[Pact7>0])*SM1.dt
Pm7 = SM7.zeta2*np.sum(Pact7[Pact7<0])*SM1.dt

C15 = SM15.zeta1*np.sum([SM15.C_tot[:,s] for s in stimlocs_15],axis=1).mean()*SM1.dt
Pact15  = np.mean([SM15.P_tot[:,s]*SM15.F(SM15.S_tot[:,s]) for s in stimlocs_15],axis=0)
Pp15 = SM15.zeta2*np.sum(Pact15[Pact15>0])*SM1.dt
Pm15 = SM15.zeta2*np.sum(Pact15[Pact15<0])*SM1.dt

p1 =plt.bar([1,2,3,4],[Pp1+C1,Pp3+C3,
                   Pp7+C7,Pp15+C15]
       ,label = r'$C_d$',color=C[ColorDict['Control']])
p2 =plt.bar([1,2,3,4],[Pp1+C1-SM3.C_s,Pp3+C3-SM3.C_s,
                   Pp7+C7-SM7.C_s,Pp15+C15-SM15.C_s]
       ,label = r'$C_d$',color=C[ColorDict['Control']-1])
p3 =plt.bar([1,2,3,4],[Pp1*100,Pp3*100,
                   Pp7*100,Pp15*100]
       ,label = r'I$C_d$',color='r')

p4 =plt.bar([1,2,3,4],[Pm1,Pm3,
                   Pm7,Pm15]
       ,label = r'Init. $C_d$',color=C[ColorDict['CHX']-1])
plt.axhline(0,ls='--',lw=3,c='k',alpha=0.5)
plt.yticks([-0.75,2.5],['Shrinking','Growing'],rotation=90)
plt.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    left=False,        # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) 
plt.xticks([1,2,3,4],['1 Spine', '3 Spine','7 Spine','15 Spine'],rotation=25)
ax = plt.gca()
l1 = plt.legend([p1,p2], [r"$C_s$",r'$C_d$'], loc=1,framealpha=0.5,fontsize=24)
l2 = plt.legend([p3,p4], [r'$P+$',r'$P-$'], loc=4,framealpha=0.5,fontsize=24,
                bbox_to_anchor=(1.05,0))
ax.add_artist(l1)
l1.get_frame().set_linewidth(0.0)
l2.get_frame().set_linewidth(0.0)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### d

In [ ]:
print(wR2(Control[3:],expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(Control[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))
print(NMSE(Control[3:],expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(Control[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))

In [ ]:
print(wR2(mtot3[3:],expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot3[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))
print(NMSE(mtot3[3:],expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot3[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))

In [ ]:
print(wR2(mt7[:,3:].T,expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mt7.T[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))
print(NMSE(mt7[:,3:].T,expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mt7.T[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))

In [ ]:
print(wR2(mtot15[3:],expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot15[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))
print(NMSE(mtot15[3:],expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot15[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])))


In [ ]:
Func = expfuncArr(np.linspace(2,40),sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(Control[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])
Func2 = expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(Control[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])
plt.plot(np.linspace(2,40),Func,
         c=C[ColorDict['Control']],lw=5,
         label='1 Spine, \n $R_w^2$ = {} \n NMSE = {}'.format(wR2(Control[3:],Func2),NMSE(Control[3:],Func2)))
Func = expfuncArr(np.linspace(2,40),sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot3[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])
Func2 = expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot3[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])
plt.plot(np.linspace(2,40),Func,lw=5,c=C[ColorDict['Control']]+0.15,
         label='3 Spine \n $R_w^2$ = {} \n NMSE = {}'.format(wR2(mtot3[3:],Func2),NMSE(mtot3[3:],Func2)),ls='--')
Func = expfuncArr(np.linspace(2,40),sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mt7.T[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])
Func2 = expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mt7.T[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])

plt.plot(np.linspace(2,40),Func,lw=5,c=np.clip(C[ColorDict['Control']]+0.3,0,1),
        label='7 Spine\n $R_w^2$ = {} \n NMSE = {}'.format(wR2(mt7.T[3:],Func2),NMSE(mt7.T[3:],Func2)),ls='-.')

Func = expfuncArr(np.linspace(2,40),sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot15[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])
Func2 = expfuncArr(SM7.tvec,sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mtot15[3:],axis=1),bounds=[(0,0,0),(10,10,10)],maxfev=5000)[0])

plt.plot(np.linspace(2,40),Func,lw=5,c=np.clip(C[ColorDict['Control']]+0.45,0,1),         
         label='15 Spine\n $R_w^2$ = {} \n NMSE = {}'.format(wR2(mtot15[3:],Func2),NMSE(mtot15[3:],Func2)))
plt.legend(fontsize=20,loc=1,frameon=False,ncols=2,columnspacing=0.3,handlelength=1.1,handletextpad=0.2,
           bbox_to_anchor=(1.1,1))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([2,10,20,30,40])
plt.ylim((0.9076591483301499, 3.006948551593322))
plt.xlim((-2.0, 42.0))

### e

In [ ]:
SpineDir = DatDir+'/1Spine/'
mt= LoadOneSpine(SpineDir,False)
mt1 = np.copy(mt)

SpineDir = DatDir+'3Spine/'
m,_ = LoadThreeSpine(SpineDir,True)
mtot = np.hstack(m)
mtot3 = np.copy(mtot)

SpineDir = DatDir+'7Spine/'
m,_ = LoadSevenSpine(SpineDir,True)
mtot = np.hstack(m)
mtot7 = np.copy(mtot)

SpineDir = DatDir+'15Spine/'
m,_ = Load15Spine(SpineDir,True)
mtot = np.hstack(m)
mtot15 = np.copy(mtot)


In [ ]:
def bootstrap_curve_fit(dat,Num):
    bootExp = np.zeros((Num,3))
    for k in tqdm(range(Num)):
        mboot = dat[:,np.random.choice(range(dat.shape[1]),mt1.shape[1]//2,replace=True)]
        try:
            val = sp.optimize.curve_fit(expfunc,[2,10,20,30,40],np.nanmean(mboot[3:],axis=1),maxfev=5000,bounds=[(-np.inf,0,0),(np.inf,5,5)])[0]
            bootExp[k] = val
        except Exception as e:
            print(e)
    return bootExp
def compute_confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    std_err = np.std(data, ddof=1) / np.sqrt(len(data))
    h = std_err * norm.ppf((1 + confidence) / 2)
    return mean,mean - h, mean + h

In [ ]:
np.random.seed(0)
n_iter  = 1000
bootExp1 = bootstrap_curve_fit(mt1, Num=n_iter)

bootExp3 = bootstrap_curve_fit(mtot3, Num=n_iter)

bootExp7 = bootstrap_curve_fit(mtot7, Num=n_iter)

bootExp15 = bootstrap_curve_fit(mtot15, Num=n_iter)

b1_a,b1_b,b1_c = bootExp1[:,0],bootExp1[:,1],bootExp1[:,2]
b3_a,b3_b,b3_c = bootExp3[:,0],bootExp3[:,1],bootExp3[:,2]
b7_a,b7_b,b7_c = bootExp7[:,0],bootExp7[:,1],bootExp7[:,2]
b15_a,b15_b,b15_c = bootExp15[:,0],bootExp15[:,1],bootExp15[:,2]

In [ ]:

boot_results = [b1_a, b3_a, b7_a, b15_a]
box = plt.boxplot(boot_results,whis=[2.5,97.5],showfliers=False,patch_artist=True,showcaps=False,meanline=True,)
plt.plot([1,2,3,4],[b.mean() for b in boot_results],'^k',ms=7)
plt.xticks([1, 2, 3, 4], ['1', '3', '7', '15'])
colors = [C[ColorDict['Control']],np.clip(C[ColorDict['Control']]+0.15,0,1),np.clip(C[ColorDict['Control']]+0.3,0,1),np.clip(C[ColorDict['Control']]+0.45,0,1)]
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_edgecolor(color)

for whisker in box['whiskers']:
    whisker.set_color('black')
    whisker.set_linewidth(2)
    whisker.set_alpha(0.5)


for means in box['means']:
    means.set_color('black')
    means.set_linewidth('2')

plt.plot([1,4],[58,58],'k')
plt.plot(2.5,61,marker=(6, 2, 0),c='k',ms=15)

# Post-hoc test using permutation test
ps = skp.posthoc_dunn(boot_results, p_adjust='fdr_bh')

# Print the pairwise p-values matrix
print(ps)

### f

In [ ]:
boot_results = [b1_b, b3_b, b7_b, b15_b]
box = plt.boxplot(boot_results,whis=[5,95],showfliers=False,patch_artist=True,showcaps=False,meanline=True,)
plt.plot([1,2,3,4],[b.mean() for b in boot_results],'^k',ms=7)
plt.xticks([1, 2, 3, 4], ['1', '3', '7', '15'])
colors = [C[ColorDict['Control']],np.clip(C[ColorDict['Control']]+0.15,0,1),np.clip(C[ColorDict['Control']]+0.3,0,1),np.clip(C[ColorDict['Control']]+0.45,0,1)]
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_edgecolor(color)

for whisker in box['whiskers']:
    whisker.set_color('black')
    whisker.set_linewidth(2)
    whisker.set_alpha(0.5)


for means in box['means']:
    means.set_color('black')
    means.set_linewidth('2')

# Post-hoc test using permutation test
ps = skp.posthoc_dunn(boot_results, p_adjust='fdr_bh')

plt.plot(2,3.,marker=f'($*$)',c='k',ms=20)
plt.plot(3,3.,marker=f'($*$)',c='k',ms=20)
plt.plot(1,3.,marker=f'($\dag$)',c='k',ms=20)
plt.plot(4,3.,marker=f'($\dag$)',c='k',ms=20)


# Print the pairwise p-values matrix
print(ps)


### g

In [ ]:
boot_results = [b1_c, b3_c, b7_c, b15_c]
box = plt.boxplot(boot_results,whis=[5,95],showfliers=False,patch_artist=True,showcaps=False,meanline=True,)
plt.plot([1,2,3,4],[b.mean() for b in boot_results],'^k',ms=7)
plt.xticks([1, 2, 3, 4], ['1', '3', '7', '15'])
colors = [C[ColorDict['Control']],np.clip(C[ColorDict['Control']]+0.15,0,1),np.clip(C[ColorDict['Control']]+0.3,0,1),np.clip(C[ColorDict['Control']]+0.45,0,1)]
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_edgecolor(color)

for whisker in box['whiskers']:
    whisker.set_color('black')
    whisker.set_linewidth(2)
    whisker.set_alpha(0.5)


for means in box['means']:
    means.set_color('black')
    means.set_linewidth('2')

plt.plot(1,2.3,marker=f'($*$)',c='k',ms=20)
plt.plot(2,2.3,marker=f'($\dag$)',c='k',ms=20)
plt.plot(3,2.3,marker=f'($\dag$)',c='k',ms=20)
plt.plot(4,2.3,marker=f'($\ddag$)',c='k',ms=20)




# Post-hoc test using permutation test
ps = skp.posthoc_dunn(boot_results, p_adjust='fdr_bh')

# Print the pairwise p-values matrix
print(ps)

### h

In [ ]:
dat = [mtot3[3],mtot7[3],mtot15[3]]
NMSE2 = []
for i,t in enumerate([3,7,15]):
    p = func(t,4.13957397, 1.29458818)
    y = dat[i]
    NMSE2.append((sum((p-y)**2)/(sum((y-y.mean())**2)/len(y)))/len(y))
print(np.round(np.mean(NMSE2),3))

y    = [np.mean(d) for d in dat]
yvar = [np.var(d) for d in dat]
ymean = np.mean(y)
sum1 = 0
sum2 = 0
for i,t in enumerate([3,7,15]):
    p = func(t,4.13957397, 1.29458818)
    sum1 +=  (p-y[i])**2/yvar[i]
    sum2 +=  (ymean-y[i])**2/yvar[i]
print(np.round(1-sum1/sum2,3))

In [ ]:
vals = np.hstack([mtot3[3],mtot7[3],mtot15[3]])

In [ ]:
params = sp.optimize.curve_fit(func,[3]*len(mtot3[3])+[7]*len(mtot7[3])+len(mtot15[3])*[15],vals)[0]
Func = func([3,7,15],params[0],params[1])

#wR2(np.array([mtot3[3],mtot7[3],mtot15[3]]),Func)


In [ ]:
mtot3.shape

In [ ]:
SpineDir = DatDir+'/1Spine/'
mt= LoadOneSpine(SpineDir,False)
mt1 = np.copy(mt)

SpineDir = DatDir+'3Spine/'
m,_ = LoadThreeSpine(SpineDir,True)
mtot = np.hstack(m)
mtot3 = np.copy(mtot)

SpineDir = DatDir+'7Spine/'
m,_ = LoadSevenSpine(SpineDir,True)
mtot = np.hstack(m)
mtot7 = np.copy(mtot)

SpineDir = DatDir+'15Spine/'
m,_ = Load15Spine(SpineDir,True)
mtot = np.hstack(m)
mtot15 = np.copy(mtot)

plt.bar([1],np.mean(mt1[3]),yerr=stats.sem(mt1[3]),color=C[ColorDict['Control']],width=1.5)
plt.scatter(1+0.2*np.random.randn(*mt1[3].shape),mt1[3],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([3],np.mean(mtot3[3]),yerr=stats.sem(mtot3[3]),color=np.clip(C[ColorDict['Control']]+0.15,0,1),width=1.5)
plt.scatter(3+0.2*np.random.randn(*mtot3[3].shape),mtot3[3],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([7],np.mean(mtot7[3]),yerr=stats.sem(mtot7[3]),color=np.clip(C[ColorDict['Control']]+0.3,0,1),width=1.5)
plt.scatter(7+0.2*np.random.randn(*mtot7[3].shape),mtot7[3],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([15],np.mean(mtot15[3]),yerr=stats.sem(mtot15[3]),color=np.clip(C[ColorDict['Control']]+0.45,0,1),width=1.5)
plt.scatter(15+0.2*np.random.randn(*mtot15[3].shape),mtot15[3],facecolors='none', edgecolors='k',alpha=0.5)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

PlotSig(3,7,3.2,0.15)

PlotSig(7,15,2.2,0.15,stars=2,starpad=0.5)
ax.set_xticks([1,3,7,15],[1,3,7,15],fontsize=32)
plt.plot(np.linspace(3,15,100),func(np.linspace(3,15,100),4.13957397, 1.29458818),lw=3,c='r',
         label=r'1/$N$ fit',ls='--')
leg = ax.legend(framealpha=0.0,fontsize=28,loc=1,bbox_to_anchor=(1.0,1.15),frameon=False)
plt.text(8.,4,'NMSE=1.001 \n $R_w^2$=0.998',fontsize=28)

In [ ]:
stats.ttest_ind(mtot15[3],mtot7[3])

### i

In [ ]:
SpineDir = DatDir+'3Spine/'
m,d,p = LoadSevenSpine3(SpineDir,False)
x3 = []
dist = 0
for d1 in d:
    x3.append(sum(m[dist:len(d1)+dist,3]))
    dist += len(d1)
SpineDir = DatDir+'7Spine/'
m,d,p = LoadSevenSpine3(SpineDir,False)
x7 = []
dist = 0
for d1 in d:
    x7.append(sum(m[dist:len(d1)+dist,3]))
    dist += len(d1)
SpineDir = DatDir+'15Spine/'
m,d,p = LoadSevenSpine3(SpineDir,False)
x15 = []
dist = 0
for d1 in d:
    x15.append(sum(m[dist:len(d1)+dist,3]))
    dist += len(d1)

In [ ]:
xt = [1]*len(mt1[3])+[3]*len(x3)+[7]*len(x7)+len(x15)*[15]

In [ ]:
sp.optimize.curve_fit(logfunc,xt,np.hstack([mt1[3],x3,x7,x15]))

In [ ]:
sp.optimize.curve_fit(powerfunc,xt,np.hstack([mt1[3],x3,x7,x15]))

In [ ]:
sp.stats.linregress(xt,np.hstack([mt1[3],x3,x7,x15]))

In [ ]:
dat = [mt1[3],x3,x7,x15]
NMSE2 = []
for i,t in enumerate([1,3,7,15]):
    p = t*1.1242942155516449 +2.331687634292547
    y = np.array(dat[i])
    NMSE2.append((sum((p-y)**2)/(sum((y-y.mean())**2)/len(y)))/len(y))
print(np.round(np.mean(NMSE2),3))

y    = [np.mean(d) for d in dat]
yvar = [np.var(d) for d in dat]
ymean = np.mean(y)
sum1 = 0
sum2 = 0
for i,t in enumerate([1,3,7,15]):
    p = t*1.1242942155516449 +2.331687634292547
    sum1 +=  (p-y[i])**2/yvar[i]
    sum2 +=  (ymean-y[i])**2/yvar[i]
print(np.round(1-sum1/sum2,3))


In [ ]:
dat = [mt1[3],mtot3[3]*3,mtot7[3]*7,mtot15[3]*15]
NMSE2 = []
for i,t in enumerate([1,3,7,15]):
    p = powerfunc(t,7.30690351,  0.4328226 , -5.49327683)
    y = np.array(dat[i])
    NMSE2.append((sum((p-y)**2)/(sum((y-y.mean())**2)/len(y)))/len(y))
print(np.round(np.mean(NMSE2),3))

y    = [np.mean(d) for d in dat]
yvar = [np.var(d) for d in dat]
ymean = np.mean(y)
sum1 = 0
sum2 = 0
for i,t in enumerate([1,3,7,15]):
    p = powerfunc(t,7.30690351,  0.4328226 , -5.49327683)
    sum1 +=  (p-y[i])**2/yvar[i]
    sum2 +=  (ymean-y[i])**2/yvar[i]
print(np.round(1-sum1/sum2,3))


In [ ]:
dat = [mt1[3],mtot3[3]*3,mtot7[3]*7,mtot15[3]*15]
NMSE2 = []
for i,t in enumerate([1,3,7,15]):
    p = logfunc(t,11.64117937,  0.25304444,  0.92073568)
    y = np.array(dat[i])
    NMSE2.append((sum((p-y)**2)/(sum((y-y.mean())**2)/len(y)))/len(y))
print(np.round(np.mean(NMSE2),3))

y    = [np.mean(d) for d in dat]
yvar = [np.var(d) for d in dat]
ymean = np.mean(y)
sum1 = 0
sum2 = 0
for i,t in enumerate([1,3,7,15]):
    p = logfunc(t,11.64117937,  0.25304444,  0.92073568)
    sum1 +=  (p-y[i])**2/yvar[i]
    sum2 +=  (ymean-y[i])**2/yvar[i]
    print(sum1/sum2)
print(np.round(1-sum1/sum2,3))



In [ ]:
x = np.linspace(1,15,100)
plt.bar([1],(np.mean(mt1[3])),yerr=stats.sem(mt1[3]),color=C[ColorDict['Control']],width=1.5)
plt.scatter(1+0.2*np.random.randn(*mt1[3].shape),mt1[3],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([3],np.mean(x3),yerr=stats.sem(x3),color=np.clip(C[ColorDict['Control']]+0.15,0,1),width=1.5)
plt.scatter(3+0.2*np.random.randn(len(x3)),x3,facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([7],np.mean(x7),yerr=stats.sem(x7),color=np.clip(C[ColorDict['Control']]+0.3,0,1),width=1.5)
plt.scatter(7+0.2*np.random.randn(len(x7)),x7,facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([15],np.mean(x15),yerr=stats.sem(x15),color=np.clip(C[ColorDict['Control']]+0.45,0,1),width=1.5)
plt.scatter(15+0.2*np.random.randn(len(x15)),x15,facecolors='none', edgecolors='k',alpha=0.5)
plt.plot(x,x*1.1242942155516449 +2.331687634292547,'g',lw=3,label='Linear fit \n $m = 1.36$')
plt.plot(x,powerfunc(x,7.30690351,  0.4328226 , -5.49327683),'k',ls='--',lw=3,label='Power law \n $b = 0.62$')
plt.plot(x,logfunc(x,11.64117937,  0.25304444,  0.92073568 ),'r',lw=3,ls='-.',label='Logarithm')

plt.xticks([1,3,7,15])
plt.legend(frameon=False,ncol=3,loc = 1,bbox_to_anchor=(1.1,1.3),fontsize=28,columnspacing=0.3,handlelength=0.6,handletextpad=0.1)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

plt.text(0,21,'NMSE=3.418',fontsize=24,c='g')
plt.text(0,19,'$R_w^2$=0.949',fontsize=24,c='g')

plt.text(0,15,'NMSE=1.329',fontsize=24,c='k')
plt.text(0,13,'$R_w^2$=0.996',fontsize=24,c='k')

plt.text(8,19,'NMSE=1.358 \n $R_w^2$=0.995',fontsize=24,c='r')

### j

In [ ]:
xt = [1]*len(mt1[3])+[3]*len(mtot3[-1])+[7]*len(mtot7[-1])+len(mtot15[-1])*[15]

In [ ]:
sp.stats.linregress(xt,np.hstack([mt1[-1],mtot3[-1],mtot7[-1],mtot15[-1]]))

In [ ]:
dat = [mt1[-1],mtot3[-1],mtot7[-1],mtot15[-1]]
NMSE2 = []
for i,t in enumerate([1,3,7,15]):
    p = t*0.013199135932088976+1.230170493013704
    y = dat[i]
    NMSE2.append((sum((p-y)**2)/(sum((y-y.mean())**2)/len(y)))/len(y))
print(np.round(np.mean(NMSE2),3))

y    = [np.mean(d) for d in dat]
yvar = [np.var(d) for d in dat]
ymean = np.mean(y)
sum1 = 0
sum2 = 0
for i,t in enumerate([1,3,7,15]):
    p = t*0.013199135932088976+1.230170493013704
    sum1 +=  (p-y[i])**2/yvar[i]
    sum2 +=  (ymean-y[i])**2/yvar[i]
print(np.round(1-sum1/sum2,3))


In [ ]:
SpineDir = DatDir+'/1Spine/'
mt= LoadOneSpine(SpineDir,True)
mt1 = np.copy(mt)

SpineDir = DatDir+'3Spine/'
m,_ = LoadThreeSpine(SpineDir,True)
mtot = np.hstack(m)
mtot3 = np.copy(mtot)

SpineDir = DatDir+'7Spine/'
m,_ = LoadSevenSpine(SpineDir,True)
mtot = np.hstack(m)
mtot7 = np.copy(mtot)

SpineDir = DatDir+'15Spine/'
m,_ = Load15Spine(SpineDir,True)
mtot = np.hstack(m)
mtot15 = np.copy(mtot)

In [ ]:
plt.bar([1],np.mean(Control[-1]),yerr=stats.sem(mt1[-1]),color=C[ColorDict['Control']],width=1.5)
plt.scatter(1+0.2*np.random.randn(*Control[-1].shape),Control[-1],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([3],np.mean(mtot3[-1]),yerr=stats.sem(mtot3[-1]),color=C[ColorDict['Control']]+0.15,width=1.5)
plt.scatter(3+0.2*np.random.randn(*mtot3[-1].shape),mtot3[-1],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([7],np.mean(mtot7[-1]),yerr=stats.sem(mtot7[-1]),color=np.clip(C[ColorDict['Control']]+0.3,0,1),width=1.5)
plt.scatter(7+0.2*np.random.randn(*mtot7[-1].shape),mtot7[-1],facecolors='none', edgecolors='k',alpha=0.5)
plt.bar([15],np.mean(mtot15[-1]),yerr=stats.sem(mtot15[-1]),color=np.clip(C[ColorDict['Control']]+0.45,0,1),width=1.5)
plt.scatter(15+0.2*np.random.randn(*mtot15[-1].shape),mtot15[-1],facecolors='none', edgecolors='k',alpha=0.5)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([1,3,7,15],[1,3,7,15])
plt.plot([1,3,7,15],np.array([1,3,7,15])*0.013199135932088976+1.230170493013704,lw=3,c='r',ls='--',
        label='Linear fit, $m = 0.013$')
leg = ax.legend(framealpha=0.5,fontsize=36,loc=4,bbox_to_anchor=(1.1,0.9),frameon=False)
plt.text(1,3,'NMSE=1.002, $R_w^2$=0.888',fontsize=24)

## Supplemental figure 1

In [ ]:
params =  [1.63241027e-03, 1.05057645e+00, 4.79773558e-03,1.20091169e+00,
     1.19406065e-01, 4.49843716e-02 , 0.2 ,2.01143310e+00,
     10, 1.00000000e+00,0.28093303e+00, 2.07116490e+00,
     3.01751859e+00, 7.94972648e+00] #3.3 works with rho (one) and C (seven)

a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_three = np.array([[-0.19,0,0.19],[0,0,0]])

SpineWidth = 1000
SM32 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM32.Simulate()
xvec = SM32.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]


### a

In [ ]:
params =  [1.63241027e-03, 1.05057645e+00, 4.79773558e-03,1.20091169e+00,
     1.19406065e-01, 4.49843716e-02 , 0.2 ,2.01143310e+00,
     10, 1.00000000e+00,0.28093303e+00, 2.07116490e+00,
     3.01751859e+00, 7.94972648e+00] #3.3 works with rho (one) and C (seven)

a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_three = np.array([[-0.19,0,0.19],[0,0,0]])

SpineWidth = 1000
SM32 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM32.Simulate()
xvec = SM32.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]


In [ ]:
xmu = np.linspace(-14,14,400)
plt.plot(xmu,SM3.C_tot[0,:],lw=3,c='k',label='Init. C')

plt.axvline(xmu[200],lw=5,label='Stimulated spine')
plt.axvline(xmu[210],lw=5,c=C[ColorDict['Control']-1],label='Inside cluster')
plt.axvline(xmu[230],lw=5,c=C[ColorDict['CHX']],label='Outside cluster')
plt.axvline(xmu[250],lw=5,c=C[ColorDict['CHX']-1],label='Away from cluster')
plt.xlim([-5,5])
plt.plot([xmu[175],xmu[186]],[0.1,0.1],c='gray',lw=3)
plt.text(xmu[148],0.06,r'\bf{1}$\mathbf{\mu m}$',color='gray',fontsize=32)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### b

In [ ]:
plt.plot(SM3.tvec[:400],SM3.S_tot[:400,200],c=C[ColorDict['Control']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.S_tot[:400,190],c=C[ColorDict['Control']-1],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.S_tot[:400,170],c=C[ColorDict['CHX']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.S_tot[:400,150],c=C[ColorDict['CHX']],lw=1,ls='--')

plt.plot(SM3.tvec[400:],SM3.S_tot[400:,200],c=C[ColorDict['Control']],lw=3,label='Stimulated spine')
plt.plot(SM3.tvec[400:],SM3.S_tot[400:,190],c=C[ColorDict['Control']-1],lw=3,label='Inside cluster')
plt.plot(SM3.tvec[400:],SM3.S_tot[400:,170],c=C[ColorDict['CHX']],lw=3,label='Outside cluster')
plt.plot(SM3.tvec[400:],SM3.S_tot[400:,150],c=C[ColorDict['CHX']-1],lw=3,label='Away from cluster')
ax = plt.gca()
leg = ax.legend(framealpha=0.5,fontsize=28,loc=1,bbox_to_anchor=(1.1,1.0))
leg.get_frame().set_linewidth(0.0)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([1,2],[1,2])


### c

In [ ]:
plt.plot(SM3.tvec[:400],SM3.S_tot[:400,200],c=C[ColorDict['Control']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM32.S_tot[:400,190],c=C[ColorDict['Control']-1],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM32.S_tot[:400,170],c=C[ColorDict['CHX']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM32.S_tot[:400,150],c=C[ColorDict['CHX']],lw=1,ls='--')

plt.plot(SM3.tvec[400:],SM3.S_tot[400:,200],c=C[ColorDict['Control']],lw=3,label='Stimulated spine')
plt.plot(SM3.tvec[400:],SM32.S_tot[400:,190],c=C[ColorDict['Control']-1],lw=3,label='Inside cluster')
plt.plot(SM3.tvec[400:],SM32.S_tot[400:,170],c=C[ColorDict['CHX']],lw=3,label='Outside cluster')
plt.plot(SM3.tvec[400:],SM32.S_tot[400:,150],c=C[ColorDict['CHX']-1],lw=3,label='Away from cluster')
plt.text(28,2.5,'Heterosynaptic \n threshold lower',ha='center', va='center',fontsize=32)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


### d

In [ ]:
plt.plot(SM3.tvec[:400],SM3.C_tot[:400,200],c=C[ColorDict['Control']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.C_tot[:400,190],c=C[ColorDict['Control']-1],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.C_tot[:400,170],c=C[ColorDict['CHX']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.C_tot[:400,150],c=C[ColorDict['CHX']],lw=1,ls='--')

plt.plot(SM3.tvec[400:],SM3.C_tot[400:,200],c=C[ColorDict['Control']],lw=3,label='Stimulated spine')
plt.plot(SM3.tvec[400:],SM3.C_tot[400:,190],c=C[ColorDict['Control']-1],lw=3,label='Inside cluster')
plt.plot(SM3.tvec[400:],SM3.C_tot[400:,170],c=C[ColorDict['CHX']],lw=3,label='Outside cluster')
plt.plot(SM3.tvec[400:],SM3.C_tot[400:,150],c=C[ColorDict['CHX']-1],lw=3,label='Away from cluster')
ax = plt.gca()
leg = ax.legend(framealpha=0.5,fontsize=32,loc=1,bbox_to_anchor=(1.1,1.0))
leg.get_frame().set_linewidth(0.0)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)


### e

In [ ]:
plt.plot(SM3.tvec[:400],SM3.P_tot[:400,200]*SM3.F(SM3.S_tot[:400,200]),c=C[ColorDict['Control']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.P_tot[:400,190]*SM3.F(SM3.S_tot[:400,190]),c=C[ColorDict['Control']-1],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.P_tot[:400,170]*SM3.F(SM3.S_tot[:400,170]),c=C[ColorDict['CHX']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM3.P_tot[:400,150]*SM3.F(SM3.S_tot[:400,150]),c=C[ColorDict['CHX']],lw=1,ls='--')

plt.plot(SM3.tvec[400:],SM3.P_tot[400:,200]*SM3.F(SM3.S_tot[400:,200]),c=C[ColorDict['Control']],lw=3)
plt.plot(SM3.tvec[400:],SM3.P_tot[400:,190]*SM3.F(SM3.S_tot[400:,190]),c=C[ColorDict['Control']-1],lw=3)
plt.plot(SM3.tvec[400:],SM3.P_tot[400:,170]*SM3.F(SM3.S_tot[400:,170]),c=C[ColorDict['CHX']],lw=3)
plt.plot(SM3.tvec[400:],SM3.P_tot[400:,150]*SM3.F(SM3.S_tot[400:,150]),c=C[ColorDict['CHX']-1],lw=3)
plt.axhline(0,lw=3,c='k',ls='--')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([0,-0.02,-0.04],[0,-0.02,-0.04],rotation=90,va='center')
plt.text(28,-0.02,'Unchanged \n parameters',ha='center', va='center',fontsize=32)


### f

In [ ]:
plt.plot(SM3.tvec[:400],SM3.P_tot[:400,200]*SM3.F(SM3.S_tot[:400,200]),c=C[ColorDict['Control']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM32.P_tot[:400,190]*SM32.F(SM32.S_tot[:400,190]),c=C[ColorDict['Control']-1],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM32.P_tot[:400,170]*SM32.F(SM32.S_tot[:400,170]),c=C[ColorDict['CHX']],lw=1,ls='--')
plt.plot(SM3.tvec[:400],SM32.P_tot[:400,150]*SM32.F(SM32.S_tot[:400,150]),c=C[ColorDict['CHX']],lw=1,ls='--')

plt.plot(SM3.tvec[400:],SM3.P_tot[400:,200]*SM3.F(SM3.S_tot[400:,200]),c=C[ColorDict['Control']],lw=3)
plt.plot(SM3.tvec[400:],SM32.P_tot[400:,190]*SM32.F(SM32.S_tot[400:,190]),c=C[ColorDict['Control']-1],lw=3)
plt.plot(SM3.tvec[400:],SM32.P_tot[400:,170]*SM32.F(SM32.S_tot[400:,170]),c=C[ColorDict['CHX']],lw=3)
plt.plot(SM3.tvec[400:],SM32.P_tot[400:,150]*SM32.F(SM32.S_tot[400:,150]),c=C[ColorDict['CHX']-1],lw=3)
plt.axhline(0,lw=3,c='k',ls='--')
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_yticks([0,-0.02,-0.04],[0,-0.02,-0.04],rotation=90,va='center')
plt.text(28,-0.03,'Heterosynaptic \n threshold lower',ha='center', va='center',fontsize=32)


### g

In [ ]:
params = np.load(DatDir+'ModelFit/ControlFit3.npy')

a1 = params[0]
a2 = params[1]

b1 = params[2]
b2 =  params[3]
rho = params[4]
gamma = params[5]

nu = params[6]
phi = params[7]

lam = params[8]
mu  = params[9] 
Cs  = params[10]
Cd  = params[11]

zeta1 = params[12]
zeta2 = params[13]

tMax  = 40
xMax  = 2

times = [2,10,20,30,40]
stims_three = np.array([[-0.13,0,0.13],[0,0,0]])

SpineWidth = 1000
SM3 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM3.Simulate()
xvec = SM3.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]

Min3_m = SM3.S_tot[:,stimlocs_three[1]]
Min3_e = SM3.S_tot[:,stimlocs_three[0]]

stims_three = np.array([[-0.25,0,0.25],[0,0,0]])

SpineWidth = 1000
SM3 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM3.Simulate()
xvec = SM3.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]

Max3_m = SM3.S_tot[:,stimlocs_three[1]]
Max3_e = SM3.S_tot[:,stimlocs_three[0]]

stims_three = np.array([[-0.19,0,0.19],[0,0,0]])

SpineWidth = 1000
SM3 = SpatialModel([a1,a2],[b1,b2,rho],gamma,[zeta1,zeta2],[nu,phi],[lam,mu,Cs,Cd],tMax,xMax,stims_three)
SM3.Simulate()
xvec = SM3.x.round(2)
stimlocs_three = [np.argwhere(xvec == s)[0,0] for s in stims_three[0]]

Mid3_m = SM3.S_tot[:,stimlocs_three[1]]
Mid3_e = SM3.S_tot[:,stimlocs_three[0]]


k=0
plt.plot(xmu,SM3.C_tot[k,:],lw=2,c=[0,0,1],label='T='+str(SM3.tvec[0])+' min')
k=50
plt.plot(xmu,SM3.C_tot[k,:],lw=2,c=[0.2,0,0.8],label='T='+str(SM3.tvec[50])+' min')
k=100
plt.plot(xmu,SM3.C_tot[k,:],lw=2,c=[0.4,0,0.6],label='T='+str(SM3.tvec[100])+' min')
k=200
plt.plot(xmu,SM3.C_tot[k,:],lw=2,c=[0.6,0,0.4],label='T='+str(SM3.tvec[200])+' min')
k=400
plt.plot(xmu,SM3.C_tot[k,:],lw=2,c=[0.8,0,0.2],label='T='+str(SM3.tvec[400])+' min')
k=800
plt.plot(xmu,SM3.C_tot[k,:],lw=2,c=[1.0,0,0.0],label='T='+str(SM3.tvec[800])+' min')
plt.xlim([-5,5])
plt.legend(frameon=False,ncols=3,fontsize=28,columnspacing=0.3,handlelength=0.6,handletextpad=0.1,loc=1,bbox_to_anchor=(1.1,1.35))

ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

### h

In [ ]:
k=0
plt.plot(xmu,SM3.P_tot[k,:],lw=2,c=[0,0,1],label='T='+str(SM3.tvec[0])+' min')
k=50
plt.plot(xmu,SM3.P_tot[k,:],lw=2,c=[0.2,0,0.8],label='T='+str(SM3.tvec[50])+' min')
k=100
plt.plot(xmu,SM3.P_tot[k,:],lw=2,c=[0.4,0,0.6],label='T='+str(SM3.tvec[100])+' min')
k=200
plt.plot(xmu,SM3.P_tot[k,:],lw=2,c=[0.6,0,0.4],label='T='+str(SM3.tvec[200])+' min')
k=400
plt.plot(xmu,SM3.P_tot[k,:],lw=2,c=[0.8,0,0.2],label='T='+str(SM3.tvec[400])+' min')
k=800
plt.plot(xmu,SM3.P_tot[k,:],lw=2,c=[1.0,0,0.0],label='T='+str(SM3.tvec[800])+' min')
plt.xlim([-5,5])
plt.legend(frameon=False,ncols=3,fontsize=28,columnspacing=0.3,handlelength=0.6,handletextpad=0.1,loc=1,bbox_to_anchor=(1.1,1.35))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)

# Supplemental Figure 2

In [ ]:
np.random.seed(43)

Colors = [
'#fdbb84',
'#fc8d59',
'#e34a33',
'#b30000',
'#a6bddb',
'#74a9cf',
'#2b8cbe',
'#045a8d',
'#fa9fb5',
'#f768a1',
'#c51b8a',
'#7a0177',
'#addd8e',
'#78c679',
'#31a354',
'#006837',]

plt.plot(0.0308,(np.random.rand()-0.5)/100,'o',c =Colors[0], label = 'IP3R puncta$^1$')

plt.plot(0.0025,(np.random.rand()-0.5)/100,'o', c =  Colors[1],label=r'CamKII$\alpha$ - slow, medium, fast$^2$')
plt.plot(0.025,(np.random.rand()-0.5)/100,'o',c =  Colors[1])
plt.plot(0.112,(np.random.rand()-0.5)/100,'o',c =  Colors[1])
plt.plot(0.0458,(np.random.rand()-0.5)/100,'s',c = Colors[1],label=r'Average CamKII$\alpha$')
plt.plot(0.019,(np.random.rand()-0.5)/100,'o',c =  Colors[2],label=r'CamKII$\delta^2$')




plt.plot(0.04,(np.random.rand()-0.5)/100,'o',color=Colors[3],label='Actin$^3$')


plt.plot(0.022,(np.random.rand()-0.5)/100,'<',c = Colors[4],label=r'GABA rec. SU $\alpha$-1$^4$')
plt.plot(0.009,(np.random.rand()-0.5)/100,'<',c = Colors[5],label=r'GABA rec. SU $\alpha$-2$^5$')
plt.plot(0.046,(np.random.rand()-0.5)/100,'<',c = Colors[6],label=r'GABA rec. SU $\alpha$-5$^5$')

plt.plot(0.05,(np.random.rand()-0.5)/100,'^',c = Colors[7],label='AMPA rec. SU GluA1$^6$')
plt.plot(0.056,(np.random.rand()-0.5)/100,'^',c = Colors[8],label='AMPA rec. SU GluA2$^7$')

plt.plot(0.018,(np.random.rand()-0.5)/100,'^',c = Colors[9],label='Inositol 1,4,5-trisphosphate \n rec. type 1$^8$') # 
plt.plot(0.071,(np.random.rand()-0.5)/100,'^',color=Colors[10],label='Neurexin$^9$') # Nrxn
plt.plot(0.02,(np.random.rand()-0.5)/100,'^',color=Colors[11],label='Neurofascin$^{10}$')

plt.plot(0.08,(np.random.rand()-0.5)/100,'v',color=Colors[12],label='Acetylcholinesterase$^{11}$')

plt.plot(0.081,(np.random.rand()-0.5)/100,'v',color=Colors[13],label='Activated H-Ras$^{12}$')
plt.plot(0.044,(np.random.rand()-0.5)/100,'v',color=Colors[14],label=r'Activated H-Ras, M$\beta$CD$^{12}$')

plt.errorbar([b1*(1.6/0.19)], [0], xerr=[b1*(1.6/0.19)*0.2],capsize=5,c='k',marker='o')

plt.plot([b1*(1.6/0.19)], [0],'ok',label='fit model diffusion')

plt.ylim([-0.01,0.05])
plt.legend(ncols=3,frameon=False,handletextpad=0.1,columnspacing=0.1,borderpad=0.0,loc = 1,bbox_to_anchor=(1.125,0.7),fontsize=10)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.yticks([])
plt.xlabel('Diffusion coefficient ($\mu m^2 /s)$',fontsize=16)
plt.xticks(fontsize=16)


# Supplemental Figure 3

In [ ]:
Calc = pd.read_excel(DatDir+'Calcium/Calcium_imaging_data.xlsx')

In [ ]:
S1 = np.array(Calc['1x spine'])
S1sem = np.array(Calc['s.e.m.'])

S3 = np.array(Calc['3x spine'])
S3sem = np.array(Calc['s.e.m..2'])

S7 = np.array(Calc['7x spine'])
S7sem = np.array(Calc['s.e.m..4'])

S15 = np.array(Calc['15x spine'])
S15sem = np.array(Calc['s.e.m..6'])

S7d = np.array(Calc['7x distributed spine'])
S7dsem = np.array(Calc['s.e.m..8'])

S1d = np.array(Calc['1x dendrite'])
S1semd = np.array(Calc['s.e.m..1'])

S3d = np.array(Calc['3x dendrite'])
S3semd = np.array(Calc['s.e.m..3'])

S7d = np.array(Calc['7x dendrite'])
S7semd = np.array(Calc['s.e.m..5'])

S15d = np.array(Calc['15x dendrite'])
S15semd = np.array(Calc['s.e.m..7'])

S7dd = np.array(Calc['7x distributed dendrite'])
S7dsemd = np.array(Calc['s.e.m..9'])



## b

In [ ]:
plt.fill_between(x = np.arange(75),y1=S1+S1sem,y2 = S1-S1sem,alpha=0.3,color=C[ColorDict['Control']])
plt.plot(np.arange(75),S1,color=C[ColorDict['Control']],label='1 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S3+S3sem,y2 = S3-S3sem,alpha=0.3,color=np.clip(C[ColorDict['Control']]+0.3,0,1))
plt.plot(np.arange(75),S3,color=np.clip(C[ColorDict['Control']]+0.3,0,1),label='3 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S7+S7sem,y2 = S7-S7sem,alpha=0.3,color=C[ColorDict['CHX']])
plt.plot(np.arange(75),S7,color=C[ColorDict['CHX']],label='7 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S15+S15sem,y2 = S15-S15sem,alpha=0.3,color=np.clip(C[ColorDict['CHX']]+0.3,0,1))
plt.plot(np.arange(75),S15,color=np.clip(C[ColorDict['CHX']]+0.3,0,1),label='15 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S7d+S7dsem,y2 = S7d-S7dsem,alpha=0.3,color='k')
plt.plot(np.arange(75),S7d,'k',label='7 Spine \n Distr.',lw=3)
plt.legend(frameon=False,ncols=2,fontsize=24,columnspacing=0.3,handlelength=0.5,handletextpad=0.2,loc=1,bbox_to_anchor=(1.1,1.1))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([-10,0,10,20,30,40,50,60,70,80])
plt.plot([5,65],[0.6,0.6],c='orange',lw=5)
plt.xlabel('Time (s)',fontsize=32)
plt.ylabel('$\Delta F/ F$ (Spine Calcium)',fontsize=28)
plt.axhline(1,lw=2,c='k',ls='dashed')

## c

In [ ]:
plt.fill_between(x = np.arange(75),y1=S1d+S1semd,y2 = S1d-S1semd,alpha=0.3,color=C[ColorDict['Control']])
plt.plot(np.arange(75),S1d,color=C[ColorDict['Control']],label='1 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S3d+S3semd,y2 = S3d-S3semd,alpha=0.3,color=np.clip(C[ColorDict['Control']]+0.3,0,1))
plt.plot(np.arange(75),S3d,color=np.clip(C[ColorDict['Control']]+0.3,0,1),label='3 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S7d+S7semd,y2 = S7d-S7semd,alpha=0.3,color=C[ColorDict['CHX']])
plt.plot(np.arange(75),S7d,color=C[ColorDict['CHX']],label='7 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S15d+S15semd,y2 = S15d-S15semd,alpha=0.3,color=np.clip(C[ColorDict['CHX']]+0.3,0,1))
plt.plot(np.arange(75),S15d,color=np.clip(C[ColorDict['CHX']]+0.3,0,1),label='15 Spine',lw=3)
plt.fill_between(x = np.arange(75),y1=S7dd+S7dsemd,y2 = S7dd-S7dsemd,alpha=0.3,color='k')
plt.plot(np.arange(75),S7dd,'k',label='7 Spine \n Distr.',lw=3)
plt.legend(frameon=False,ncols=2,fontsize=24,columnspacing=0.3,handlelength=0.5,handletextpad=0.2,loc=1,bbox_to_anchor=(1.1,1.1))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_xticks([-10,0,10,20,30,40,50,60,70,80])
plt.plot([5,65],[0.6,0.6],c='orange',lw=5)
plt.xlabel('Time (s)',fontsize=32)
plt.ylabel('$\Delta F/ F$ (Dendritic Calcium)',fontsize=28)
plt.axhline(1,lw=2,c='k',ls='dashed')

## d

In [ ]:
Calc2 = pd.read_excel(
    DatDir+'Calcium/Calcium_imaging_data.xlsx',
    sheet_name='Spatial calcium profiles')

In [ ]:
S1 = np.array(Calc2['1x'])
S1sem = np.array(Calc2['s.e.m.'])
S1dist = np.array(Calc2['Distance'])

S3 = np.array(Calc2['3x'])
S3sem = np.array(Calc2['s.e.m..1'])
S3dist = np.array(Calc2['Distance.1'])

S7 = np.array(Calc2['7x'])
S7sem = np.array(Calc2['s.e.m..2'])
S7dist = np.array(Calc2['Distance.2'])

S15 = np.array(Calc2['15x'])
S15sem = np.array(Calc2['s.e.m..3'])
S15dist = np.array(Calc2['Distance.3'])

S7d = np.array(Calc2['7x distributed'])
S7dsem = np.array(Calc2['s.e.m..4'])
S7ddist = np.array(Calc2['Distance.4'])

In [ ]:
plt.fill_between(x =S1dist,y1=S1+S1sem,y2 = S1-S1sem,alpha=0.3,color=C[ColorDict['Control']])
plt.plot(S1dist,S1,color=C[ColorDict['Control']],label='1 Spine',lw=3)
plt.fill_between(x = S3dist,y1=S3+S3sem,y2 = S3-S3sem,alpha=0.3,color=np.clip(C[ColorDict['Control']]+0.3,0,1))
plt.plot(S3dist,S3,color=np.clip(C[ColorDict['Control']]+0.3,0,1),label='3 Spine',lw=3)
plt.fill_between(x = S7dist,y1=S7+S7sem,y2 = S7-S7sem,alpha=0.3,color=C[ColorDict['CHX']])
plt.plot(S7dist,S7,color=C[ColorDict['CHX']],label='7 Spine',lw=3)
plt.fill_between(x = S15dist,y1=S15+S15sem,y2 = S15-S15sem,alpha=0.3,color=np.clip(C[ColorDict['CHX']]+0.3,0,1))
plt.plot(S15dist,S15,color=np.clip(C[ColorDict['CHX']]+0.3,0,1),label='15 Spine',lw=3)
plt.fill_between(x = S7ddist,y1=S7d+S7dsem,y2 = S7d-S7dsem,alpha=0.3,color='k')
plt.plot(S7ddist,S7d,'k',label='7 Spine - Distr.',lw=3)
plt.legend(frameon=False,ncols=3,fontsize=24,columnspacing=0.3,handlelength=0.5,handletextpad=0.2,loc=1,bbox_to_anchor=(1.1,1.3))
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
plt.xticks([-40,-30,-20,-10,0,10,20,30,40],[-40,-30,-20,-10,0,10,20,30,40],fontsize=28)
plt.xlabel('Distance along dendrite ($\mu m$)',fontsize=32)
plt.ylabel('GCaMP signal ($\Delta F/ F$)',fontsize=28)
plt.axhline(1,lw=2,c='k',ls='dashed')
plt.axvline(0,lw=2,c='k',ls='dashed')